<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fhybrid-models/models/hybrid_models_TopoRain_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
TopoRain-Net: entrenamiento y evaluación de modelos específicos por nivel de elevación.
Implementa modelos BiGRU autoencoder-decoder para cada nivel de elevación,
con fusión optimizada de características CEEMDAN y TFV-EMD usando XGBoost.
Un meta-modelo integra las predicciones de los tres modelos de elevación.
Genera métricas, scatter, mapas y tablas (global, por elevación, por percentiles).
"""

import warnings, logging
from pathlib import Path
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path
import shutil
import time
import psutil
import tensorflow as tf
import datetime
import json
from collections import defaultdict

# -----------------------------------------------------------------------------
# Configuración de logging y trazabilidad mejorada
# -----------------------------------------------------------------------------
# Crear directorio para logs
LOG_DIR = Path("logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)

# Configurar formato de timestamp
timestamp_format = "%Y-%m-%d_%H-%M-%S"
run_timestamp = datetime.datetime.now().strftime(timestamp_format)
log_filename = f"toporain_net_run_{run_timestamp}.log"

# Configurar logging con formato detallado y salida a archivo
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_DIR / log_filename),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Clase para trazabilidad del proceso
class ProcessTracker:
    def __init__(self, name="TopoRain-NET"):
        self.name = name
        self.start_time = time.time()
        self.section_times = {}
        self.current_section = None
        self.section_start = None
        self.metrics = defaultdict(dict)
        self.resources = []
        self.checkpoints = []
        
    def start_section(self, section_name):
        """Inicia el cronómetro para una sección del proceso"""
        if self.current_section:
            self.end_section()
            
        self.current_section = section_name
        self.section_start = time.time()
        logger.info(f"▶️ INICIANDO: {section_name}")
        # Registrar recursos al inicio
        self._log_resources()
        
    def end_section(self):
        """Finaliza la sección actual y registra el tiempo transcurrido"""
        if not self.current_section:
            return
            
        elapsed = time.time() - self.section_start
        self.section_times[self.current_section] = elapsed
        logger.info(f"✓ COMPLETADO: {self.current_section} en {elapsed:.2f} segundos")
        # Registrar recursos al final
        self._log_resources()
        self.current_section = None
        
    def log_metric(self, section, metric_name, value):
        """Registra una métrica"""
        self.metrics[section][metric_name] = value
        logger.info(f"📊 MÉTRICA: {section} - {metric_name}: {value}")
        
    def add_checkpoint(self, description, data=None):
        """Añade un punto de control con datos opcionales"""
        checkpoint = {
            'timestamp': time.time(),
            'description': description,
            'elapsed_total': time.time() - self.start_time,
            'data': data
        }
        self.checkpoints.append(checkpoint)
        logger.info(f"🔖 CHECKPOINT: {description}")
        
    def _log_resources(self):
        """Registra el uso de recursos actual"""
        mem_info = get_memory_info()
        cpu_percent = psutil.cpu_percent(interval=0.1)
        
        # Obtener información de GPU si está disponible
        gpu_info = get_gpu_memory_info()
        gpu_usage = None
        if gpu_info and gpu_info[0]['memory_used_mb'] > 0:
            gpu_usage = {
                'used_mb': gpu_info[0]['memory_used_mb'],
                'total_mb': gpu_info[0]['memory_total_mb'],
                'percent': gpu_info[0]['memory_used_percent']
            }
        
        resources = {
            'timestamp': time.time(),
            'memory_used_gb': mem_info['total_gb'] - mem_info['free_gb'],
            'memory_total_gb': mem_info['total_gb'],
            'memory_percent': mem_info['used_percent'],
            'cpu_percent': cpu_percent,
            'gpu': gpu_usage
        }
        self.resources.append(resources)
        
    def _convert_numpy_types(self, obj):
        """
        Convierte recursivamente tipos de numpy a tipos nativos de Python
        para hacer el objeto JSON serializable
        """
        import numpy as np
        
        if isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif isinstance(obj, (np.bool_)):
            return bool(obj)
        elif isinstance(obj, dict):
            return {key: self._convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._convert_numpy_types(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(self._convert_numpy_types(item) for item in obj)
        else:
            return obj
        
    def summary(self):
        """Genera un resumen del proceso"""
        total_time = time.time() - self.start_time
        
        # Calcular estadísticas de recursos
        if self.resources:
            avg_mem = sum(r['memory_percent'] for r in self.resources) / len(self.resources)
            max_mem = max(r['memory_percent'] for r in self.resources)
            avg_cpu = sum(r['cpu_percent'] for r in self.resources) / len(self.resources)
            max_cpu = max(r['cpu_percent'] for r in self.resources)
        else:
            avg_mem = max_mem = avg_cpu = max_cpu = 0
        
        summary_dict = {
            'name': self.name,
            'total_time': total_time,
            'start_time': self.start_time,
            'end_time': time.time(),
            'section_times': self.section_times,
            'metrics': dict(self.metrics),
            'resources': {
                'avg_memory_percent': avg_mem,
                'max_memory_percent': max_mem,
                'avg_cpu_percent': avg_cpu,
                'max_cpu_percent': max_cpu
            },
            'num_checkpoints': len(self.checkpoints)
        }
        
        # Convertir tipos numpy a tipos nativos de Python para JSON
        summary_dict = self._convert_numpy_types(summary_dict)
        
        # Guardar resumen en formato JSON
        summary_path = LOG_DIR / f"summary_{run_timestamp}.json"
        with open(summary_path, 'w') as f:
            json.dump(summary_dict, f, indent=2)
        
        logger.info(f"📑 RESUMEN DEL PROCESO GUARDADO: {summary_path}")
        
        # Imprimir resumen
        logger.info(f"📋 RESUMEN DE EJECUCIÓN - {self.name}")
        logger.info(f"  Tiempo total: {total_time:.2f} segundos")
        logger.info(f"  Secciones completadas: {len(self.section_times)}")
        for section, time_taken in sorted(self.section_times.items(), key=lambda x: x[1], reverse=True):
            logger.info(f"    - {section}: {time_taken:.2f} segundos")
        logger.info(f"  Checkpoints registrados: {len(self.checkpoints)}")
        logger.info(f"  Uso de recursos:")
        logger.info(f"    - Memoria promedio: {avg_mem:.1f}%")
        logger.info(f"    - Memoria máxima: {max_mem:.1f}%")
        logger.info(f"    - CPU promedio: {avg_cpu:.1f}%")
        logger.info(f"    - CPU máxima: {max_cpu:.1f}%")
        
        return summary_dict

# Inicializar el rastreador de procesos
tracker = ProcessTracker()

# Función para decorar funciones con trazabilidad
def trace(section_name=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            func_name = section_name or func.__name__
            tracker.start_section(func_name)
            try:
                result = func(*args, **kwargs)
                tracker.end_section()
                return result
            except Exception as e:
                logger.error(f"❌ ERROR en {func_name}: {str(e)}")
                tracker.end_section()
                raise
        return wrapper
    return decorator

# Intentar configurar el paralelismo antes de cualquier operación que inicialice el contexto
try:
    # Configurar threading para TensorFlow
    tf.config.threading.set_inter_op_parallelism_threads(4)
    tf.config.threading.set_intra_op_parallelism_threads(4)
    logger.info("Configuración de threading de TensorFlow aplicada")
except RuntimeError as e:
    # Si ya se inicializó el contexto, informar pero seguir adelante
    logger.warning(f"No se pudo configurar threading de TensorFlow: {str(e)}. Continuando con valores por defecto.")

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn ace_tools_open cartopy
    BASE_PATH = '/content/drive/MyDrive/ml_precipitation_prediction'
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

BASE = Path(BASE_PATH)
print(f"Entorno configurado. Usando ruta base: {BASE}")







FULL_NC      = BASE/"data"/"output"/"complete_dataset_with_features_with_clusters_elevation_with_windows.nc"
FUSION_NC    = BASE/"models"/"output"/"features_fusion_branches.nc"
TRAINED_DIR  = BASE/"models"/"output"/"trained_models"
TRAINED_DIR.mkdir(parents=True, exist_ok=True)
PRED_DIR = BASE/"models"/"output"/"predictions"
PRED_DIR.mkdir(parents=True, exist_ok=True)
HISTORY_DIR = BASE/"models"/"output"/"histories"
HISTORY_DIR.mkdir(parents=True, exist_ok=True)

INPUT_WINDOW   = 60
OUTPUT_HORIZON = 3

import numpy            as np
import pandas           as pd
import xarray           as xr
import geopandas        as gpd
import matplotlib.pyplot as plt
import cartopy.crs      as ccrs

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics        import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from xgboost                import XGBRegressor
# Añadir importación de LightGBM y reducción de dimensionalidad
from lightgbm               import LGBMRegressor
from sklearn.decomposition  import PCA
from sklearn.pipeline       import Pipeline

from tensorflow.keras.models    import Sequential, Model
from tensorflow.keras.layers    import Input, Dense, LSTM, GRU, Flatten, Reshape, Dropout, Concatenate, BatchNormalization, TimeDistributed, RepeatVector, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
# Importar TensorFlow aquí y configurarlo antes de cualquier operación

# Actualizar importación de mixed_precision para compatibilidad con versiones recientes de TF
try:
    # Para TensorFlow 2.4+
    from tensorflow.keras import mixed_precision
except ImportError:
    # Fallback para versiones más antiguas de TF
    from tensorflow.keras.mixed_precision import experimental as mixed_precision

import ace_tools_open as tools

# Configurar crecimiento de memoria GPU dinámico para evitar ResourceExhaustedError
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logger.info(f"GPU configurada para crecimiento dinámico de memoria: {len(gpus)} GPUs disponibles")
    except RuntimeError as e:
        logger.error(f"Error configurando GPU: {str(e)}")

# También limitar la memoria de TensorFlow para operaciones CPU
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

# Funciones auxiliares para gestión eficiente de memoria
def get_memory_info():
    """Obtiene información de memoria del sistema"""
    mem_info = psutil.virtual_memory()
    return {
        'total_gb': mem_info.total / (1024**3),
        'available_gb': mem_info.available / (1024**3),
        'used_percent': mem_info.percent,
        'free_gb': mem_info.free / (1024**3)
    }

# Funciones auxiliares para persistencia de modelos
def get_model_path(model_type, level_name, component_idx=None):
    """
    Genera la ruta para guardar o cargar un modelo específico
    
    Args:
        model_type: Tipo de modelo ('fusion', 'bigru', 'meta')
        level_name: Nombre del nivel de elevación
        component_idx: Índice de componente (para modelos de fusión)
        
    Returns:
        Path: Ruta completa del archivo del modelo
    """
    if model_type == 'fusion':
        return TRAINED_DIR / f"fusion_xgb_{level_name}_comp{component_idx}.pkl"
    elif model_type == 'bigru':
        return TRAINED_DIR / f"BiGRU_{level_name}_model.keras"
    elif model_type == 'meta':
        return TRAINED_DIR / "meta_fusion_model.pkl"
    else:
        raise ValueError(f"Tipo de modelo no reconocido: {model_type}")

def save_model(model, model_type, level_name, component_idx=None, extra_info=None):
    """
    Guarda un modelo con su información asociada
    
    Args:
        model: Modelo a guardar
        model_type: Tipo de modelo ('fusion', 'bigru', 'meta')
        level_name: Nombre del nivel de elevación
        component_idx: Índice de componente (para modelos de fusión)
        extra_info: Información adicional a guardar (pesos, métricas, etc.)
        
    Returns:
        bool: True si se guardó correctamente
    """
    try:
        model_path = get_model_path(model_type, level_name, component_idx)
        
        # Para modelos XGBoost y otros que requieren pickle
        if model_type in ['fusion', 'meta']:
            with open(model_path, 'wb') as f:
                import pickle
                data_to_save = {'model': model}
                if extra_info:
                    data_to_save['info'] = extra_info
                pickle.dump(data_to_save, f)
        
        # Para modelos Keras
        elif model_type == 'bigru':
            model.save(model_path)
            
            # Si hay info adicional, guardarla por separado
            if extra_info:
                info_path = model_path.parent / f"{model_path.stem}_info.pkl"
                with open(info_path, 'wb') as f:
                    import pickle
                    pickle.dump(extra_info, f)
        
        logger.info(f"Modelo {model_type} para {level_name} guardado en: {model_path}")
        return True
        
    except Exception as e:
        logger.error(f"Error al guardar modelo {model_type} para {level_name}: {str(e)}")
        return False

def load_model(model_type, level_name, component_idx=None):
    """
    Carga un modelo previamente guardado
    
    Args:
        model_type: Tipo de modelo ('fusion', 'bigru', 'meta')
        level_name: Nombre del nivel de elevación
        component_idx: Índice de componente (para modelos de fusión)
        
    Returns:
        model: Modelo cargado o None si no existe
        extra_info: Información adicional o None si no existe
    """
    try:
        model_path = get_model_path(model_type, level_name, component_idx)
        
        if not model_path.exists():
            return None, None
            
        # Para modelos XGBoost y otros almacenados con pickle
        if model_type in ['fusion', 'meta']:
            with open(model_path, 'rb') as f:
                import pickle
                data = pickle.load(f)
                if isinstance(data, dict) and 'model' in data:
                    model = data['model']
                    extra_info = data.get('info')
                else:
                    # Compatibilidad con formato antiguo
                    model = data
                    extra_info = None
        
        # Para modelos Keras
        elif model_type == 'bigru':
            model = tf.keras.models.load_model(model_path)
            
            # Intentar cargar info adicional si existe
            extra_info = None
            info_path = model_path.parent / f"{model_path.stem}_info.pkl"
            if info_path.exists():
                with open(info_path, 'rb') as f:
                    import pickle
                    extra_info = pickle.load(f)
        
        logger.info(f"Modelo {model_type} para {level_name} cargado desde: {model_path}")
        return model, extra_info
        
    except Exception as e:
        logger.error(f"Error al cargar modelo {model_type} para {level_name}: {str(e)}")
        return None, None

def model_exists(model_type, level_name, component_idx=None):
    """
    Verifica si existe un modelo previamente guardado
    
    Args:
        model_type: Tipo de modelo ('fusion', 'bigru', 'meta')
        level_name: Nombre del nivel de elevación
        component_idx: Índice de componente (para modelos de fusión)
        
    Returns:
        bool: True si el modelo existe
    """
    model_path = get_model_path(model_type, level_name, component_idx)
    return model_path.exists()

# Funciones auxiliares para monitorear la memoria de la GPU
def get_gpu_memory_info():
    """Obtiene la información de memoria de la GPU disponible"""
    if not gpus:
        return None

    try:
        # Intentar usar NVIDIA-SMI a través de subprocess if está disponible
        import subprocess
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used,memory.free,memory.total', '--format=csv,noheader,nounits'],
            encoding='utf-8')
        gpu_info = []
        for line in result.strip().split('\n'):
            values = [float(x) for x in line.split(',')]
            gpu_info.append({
                'memory_used_mb': values[0],
                'memory_free_mb': values[1],
                'memory_total_mb': values[2],
                'memory_used_percent': values[0] / values[2] * 100
            })
        return gpu_info
    except (ImportError, subprocess.SubprocessError, FileNotFoundError):
        # Si nvidia-smi no está disponible, usar tensorflow para obtener información limitada
        try:
            memory_info = []
            for i, gpu in enumerate(gpus):
                # En versiones nuevas de TF podemos obtener información de memoria usando experimental.VirtualDeviceConfiguration
                try:
                    mem_info = tf.config.experimental.get_memory_info(f'GPU:{i}')
                    total_memory = mem_info['current'] + mem_info['peak']  # Aproximación
                    memory_info.append({
                        'memory_used_mb': mem_info['current'] / (1024 * 1024),
                        'memory_free_mb': (total_memory - mem_info['current']) / (1024 * 1024),
                        'memory_total_mb': total_memory / (1024 * 1024),
                        'memory_used_percent': mem_info['current'] / total_memory * 100 if total_memory else 0
                    })
                except (KeyError, AttributeError, ValueError):
                    # Si no podemos obtener información específica, proveer una estimación
                    memory_info.append({
                        'memory_used_mb': -1,  # No conocido
                        'memory_free_mb': -1,  # No conocido
                        'memory_total_mb': -1,  # No conocido
                        'memory_used_percent': -1  # No conocido
                    })
            return memory_info
        except:
            return None
    return None

# Función mejorada para limpiar la memoria
def clear_memory(force_garbage_collection=True):
    """
    Limpia la memoria de manera más agresiva, liberando recursos de TensorFlow y Python

    Args:
        force_garbage_collection: Si es True, fuerza la recolección de basura
    """
    # 1. Limpiar sesión de TensorFlow para liberar variables y tensores
    try:
        tf.keras.backend.clear_session()
        logger.debug("Sesión de Keras limpiada")
    except Exception as e:
        logger.debug(f"Error al limpiar sesión de Keras: {str(e)}")

    # 2. Reiniciar gráfico de operaciones de TF si está disponible
    try:
        # Para versiones antiguas de TF que tienen reset_default_graph
        if hasattr(tf, 'reset_default_graph'):
            tf.reset_default_graph()
            logger.debug("Gráfico de TF reiniciado")
    except Exception as e:
        logger.debug(f"Error al reiniciar gráfico de TF: {str(e)}")

    # 3. Forzar recolección de basura de Python
    if force_garbage_collection:
        import gc
        # Realizar múltiples pasadas para asegurar la limpieza completa
        collected = gc.collect()
        logger.debug(f"GC recolectó {collected} objetos")

        # Segunda pasada para objetos que posiblemente se liberaron en la primera
        collected = gc.collect()
        logger.debug(f"GC recolectó {collected} objetos adicionales")

    # 4. Intentar liberar memoria al sistema operativo
    if 'psutil' in sys.modules:
        process = psutil.Process(os.getpid())
        try:
            # En Linux
            if hasattr(process, 'memory_full_info'):
                mi = process.memory_full_info()
                logger.debug(f"Memoria usada: RSS={mi.rss/1e6:.1f}MB, VMS={mi.vms/1e6:.1f}MB")

            # En sistemas POSIX, sincronizar filesystem para liberar buffers
            if hasattr(os, 'sync'):
                os.sync()
                logger.debug("Sincronizado el sistema de archivos")
        except Exception as e:
            logger.debug(f"Error en operaciones de memoria del proceso: {str(e)}")

    # 5. Intentar liberar la memoria GPU específicamente
    if gpus:
        try:
            # Ejecutar operaciones vacías para forzar sincronización de GPU
            dummy = tf.random.normal([1, 1])
            _ = dummy.numpy()  # Forzar ejecución
            logger.debug("Operaciones GPU sincronizadas")
        except Exception as e:
            logger.debug(f"Error al sincronizar GPU: {str(e)}")

# Función para crear un conjunto de datos de TensorFlow con mejor manejo de errores de memoria
def create_tf_dataset(X, Y, batch_size=32, force_cpu=False, max_retries=3):
    """
    Creates a TensorFlow Dataset from numpy arrays with batching.
    Includes robust error handling with automatic CPU fallback and batch size adjustment.

    Args:
        X: Input features array
        Y: Target labels array
        batch_size: Size of batches for training
        force_cpu: If True, forces operations to run on CPU
        max_retries: Maximum number of retry attempts with smaller batch size

    Returns:
        tf.data.Dataset object configured for training
    """
    # Verificar la memoria disponible y ajustar parámetros automáticamente
    gpu_info = get_gpu_memory_info()
    mem_info = get_memory_info()

    # Estimar si la GPU está cerca del límite (>80% usada) para decidir si forzar CPU
    auto_force_cpu = False
    if gpu_info and not force_cpu:
        for gpu in gpu_info:
            if gpu['memory_used_percent'] > 80:
                logger.warning(f"GPU usage high ({gpu['memory_used_percent']:.1f}%), forcing CPU execution")
                auto_force_cpu = True
                break

    # Si hay más de un intento, reducir el batch size
    actual_force_cpu = force_cpu or auto_force_cpu
    actual_batch_size = batch_size

    # Bucle de reintento con tamaños de batch más pequeños
    for attempt in range(max_retries):
        try:
            # Verificar si hay NaNs antes de convertir a tensores
            if np.isnan(X).any() or np.isnan(Y).any():
                logger.warning("Se detectaron NaNs en los datos. Reemplazando con ceros.")
                X = np.nan_to_num(X, nan=0.0)
                Y = np.nan_to_num(Y, nan=0.0)

            # Estrategia específica para CPU o GPU
            if actual_force_cpu:
                with tf.device('/CPU:0'):
                    # Convertir a tensores explícitamente para mejor control
                    X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
                    Y_tensor = tf.convert_to_tensor(Y, dtype=tf.float32)

                    # Crear dataset usando los tensores convertidos
                    dataset = tf.data.Dataset.from_tensor_slices((X_tensor, Y_tensor))
                    logger.info(f"Dataset creado en CPU con batch_size={actual_batch_size}")
            else:
                # Intentar crear el dataset con GPU
                dataset = tf.data.Dataset.from_tensor_slices((X, Y))
                logger.info(f"Dataset creado en GPU con batch_size={actual_batch_size}")

            # Configurar el dataset para entrenamiento con un buffer size adaptativo
            # Usar buffer size más pequeño para reducir uso de memoria
            samples = len(X)
            buffer_size = min(samples, 1000)  # Máximo 1000 elementos en memoria

            # Ajustar buffer size si la memoria está baja
            if mem_info['available_gb'] < 2.0:  # Menos de 2GB disponibles
                buffer_size = min(buffer_size, 100)  # Reducir a máximo 100 elementos
                logger.warning(f"Memoria disponible baja ({mem_info['available_gb']:.1f}GB), buffer reducido a {buffer_size}")

            dataset = dataset.shuffle(buffer_size=buffer_size, seed=42)
            dataset = dataset.batch(actual_batch_size)
            dataset = dataset.prefetch(tf.data.AUTOTUNE)

            # Probar que el dataset funciona extrayendo un batch
            try:
                for _ in dataset.take(1):
                    pass  # Solo verificar que podemos iterar
                logger.info("Dataset verificado correctamente")
            except tf.errors.ResourceExhaustedError as e:
                raise e  # Relanzar para manejar en el bloque catch

            return dataset

        except (tf.errors.ResourceExhaustedError, tf.errors.InternalError, tf.errors.FailedPreconditionError,
                tf.errors.AbortedError, tf.errors.OOM) as e:
            # Si estamos en el último intento, reducir drásticamente
            if attempt == max_retries - 1:
                logger.error(f"Error crítico al crear dataset: {str(e)}")

                # Último intento desesperado: mínimo batch size y forzar CPU
                logger.warning("Intento final con configuración mínima (batch=1, CPU)")
                with tf.device('/CPU:0'):
                    logger.info("Creando dataset final con configuración mínima")
                    # Crear con el menor batch posible
                    X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
                    Y_tensor = tf.convert_to_tensor(Y, dtype=tf.float32)
                    dataset = tf.data.Dataset.from_tensor_slices((X_tensor, Y_tensor))
                    dataset = dataset.batch(1)  # Mínimo batch size
                    return dataset
            else:
                # Reducir batch size y forzar CPU en próximo intento
                prev_batch = actual_batch_size
                actual_batch_size = max(1, actual_batch_size // 2)
                actual_force_cpu = True

                logger.warning(f"Intento {attempt+1}/{max_retries}: Reduciendo batch size de {prev_batch} a {actual_batch_size} y forzando CPU")

                # Limpiar memoria antes del próximo intento
                clear_memory()
                time.sleep(1)  # Pequeña pausa para permitir que el sistema se estabilice

# Función genérica para predecir en lotes
def predict_in_batches(model, X, batch_size=32, verbose=0):
    """
    Genera predicciones de cualquier modelo en lotes para evitar problemas de memoria

    Args:
        model: Modelo entrenado (Keras, TensorFlow, etc.)
        X: Datos de entrada (numpy array)
        batch_size: Tamaño del lote para procesamiento
        verbose: Nivel de verbosidad para las predicciones

    Returns:
        Array con predicciones
    """
    n_samples = len(X)

    # Si X es muy pequeño, predecir directamente
    if n_samples <= batch_size:
        return model.predict(X, verbose=verbose)

    # Ajustar batch_size según memoria disponible
    try:
        mem_info = get_memory_info()
        adaptive_batch = min(batch_size, max(8, int(mem_info['available_gb'] * 10)))
        logger.info(f"Generando predicciones en lotes de {adaptive_batch} muestras")
        batch_size = adaptive_batch
    except:
        # Si falla la adaptación, usar el batch_size proporcionado
        logger.info(f"Generando predicciones en lotes de {batch_size} muestras")

    # Inferir la forma de salida del modelo haciendo una predicción en un único ejemplo
    try:
        sample_pred = model.predict(X[:1], verbose=0)
        output_shape = sample_pred.shape[1:]  # Excluye la dimensión del batch
    except:
        # Si falla, asumir forma desconocida y manejarla después
        output_shape = None

    predictions = []

    # Procesar por lotes
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch_X = X[start_idx:end_idx]

        # Para mayor seguridad, comprobar si hay NaNs
        has_nans = np.isnan(batch_X).any()
        if has_nans:
            logger.warning(f"Detectados NaN en lote {start_idx}-{end_idx}, realizando imputación")
            # Reemplazar NaNs con 0 para evitar errores
            batch_X = np.nan_to_num(batch_X, nan=0.0)

        # Predecir lote
        try:
            batch_preds = model.predict(batch_X, verbose=0 if start_idx > 0 else verbose)
            predictions.append(batch_preds)

            # Liberar memoria cada 5 lotes
            if (start_idx // batch_size) % 5 == 0 and start_idx > 0:
                # Liberar memoria explícitamente
                if 'gc' in sys.modules:
                    import gc
                    gc.collect()
        except Exception as e:
            logger.error(f"Error al predecir lote {start_idx}-{end_idx}: {str(e)}")
            # Intentar con un batch más pequeño como último recurso
            try:
                smaller_batch = max(1, batch_size // 4)
                logger.warning(f"Reintentando con batch más pequeño: {smaller_batch}")
                mini_batch_preds = []
                for mini_start in range(start_idx, end_idx, smaller_batch):
                    mini_end = min(mini_start + smaller_batch, end_idx)
                    mini_X = X[mini_start:mini_end]
                    mini_pred = model.predict(mini_X, verbose=0)
                    mini_batch_preds.append(mini_pred)
                batch_preds = np.vstack(mini_batch_preds)
                predictions.append(batch_preds)
            except Exception as e2:
                logger.error(f"Error en segundo intento de lote: {str(e2)}")
                # Si también falla, rellenar con ceros
                if output_shape:
                    batch_size_curr = end_idx - start_idx
                    zeros_shape = (batch_size_curr,) + output_shape
                    logger.warning(f"Rellenando con ceros de forma {zeros_shape}")
                    predictions.append(np.zeros(zeros_shape))
                else:
                    raise e2

    # Concatenar resultados
    try:
        return np.vstack(predictions)
    except:
        # Si vstack falla (por ejemplo, formas inconsistentes), devolver una lista
        logger.warning("No se pudo concatenar predicciones, devolviendo lista de arrays")
        return predictions

# Funciones específicas para XGBoost con optimización de memoria
def predict_xgb_in_batches(model, X, batch_size=100):
    """
    Genera predicciones XGBoost en lotes para evitar problemas de memoria

    Args:
        model: Modelo XGBoost entrenado
        X: Datos de entrada (numpy array)
        batch_size: Tamaño del lote para procesamiento

    Returns:
        Array con predicciones
    """
    n_samples = len(X)
    predictions = np.zeros(n_samples)

    # Ajustar tamaño de lote según memoria disponible
    mem_info = get_memory_info()
    adaptive_batch = min(batch_size, max(10, int(mem_info['available_gb'] * 10)))

    logger.info(f"Generando predicciones XGBoost en lotes de {adaptive_batch} muestras")

    # Procesar por lotes
    for start_idx in range(0, n_samples, adaptive_batch):
        end_idx = min(start_idx + adaptive_batch, n_samples)
        batch_X = X[start_idx:end_idx]

        # Para mayor seguridad, comprobar si hay NaNs
        has_nans = np.isnan(batch_X).any()
        if has_nans:
            logger.warning(f"Detectados NaN en lote {start_idx}-{end_idx}, realizando imputación")
            # Reemplazar NaNs con 0 para evitar errores
            batch_X = np.nan_to_num(batch_X, nan=0.0)

        # Predecir lote
        batch_preds = model.predict(batch_X)
        predictions[start_idx:end_idx] = batch_preds

        # Liberar memoria cada 5 lotes
        if (start_idx // adaptive_batch) % 5 == 0 and start_idx > 0:
            if 'gc' in sys.modules:
                import gc
                gc.collect()

    return predictions

def train_xgb_with_memory_optimization(X_train, y_train, X_val=None, y_val=None, params=None):
    """
    Entrena un modelo XGBoost con optimizaciones de memoria y velocidad

    Args:
        X_train: Datos de entrenamiento
        y_train: Etiquetas de entrenamiento
        X_val: Datos de validación (opcional)
        y_val: Etiquetas de validación (opcional)
        params: Parámetros de XGBoost (diccionario)

    Returns:
        Modelo XGBoost entrenado
    """
    # Parámetros por defecto optimizados para velocidad y memoria
    default_params = {
        'n_estimators': 60,  # Reducido para mayor velocidad
        'max_depth': 4,      # Reducido para mayor velocidad
        'learning_rate': 0.2, # Aumentado para convergencia más rápida
        'subsample': 0.7,     # Reducido para mayor velocidad
        'colsample_bytree': 0.7, # Reducido para mayor velocidad
        'tree_method': 'hist',  # Método más eficiente en memoria
        'predictor': 'cpu_predictor',  # Evitar problemas de GPU
        'n_jobs': 1  # Un hilo por modelo para permitir paralelismo entre modelos
    }

    # Actualizar con parámetros personalizados si se proporcionan
    if params:
        default_params.update(params)

    # Crear y entrenar modelo con early stopping si hay datos de validación
    if X_val is not None and y_val is not None:
        model = XGBRegressor(**default_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
    else:
        # Sin early stopping si no hay datos de validación
        model = XGBRegressor(**default_params)
        model.fit(X_train, y_train)

    return model

def generate_xgb_horizon_predictions(meta_models, base_model_preds, cells, horizons=3):
    """
    Genera predicciones por horizonte usando modelos XGBoost en metamodelado

    Args:
        meta_models: Lista de modelos XGBoost (uno por horizonte)
        base_model_preds: Diccionario de predicciones de modelos base {modelo: predicciones}
        cells: Número de celdas espaciales
        horizons: Número de horizontes temporales

    Returns:
        Array de predicciones (muestras, horizontes, celdas)
    """
    # Determinar número de muestras del primer modelo base
    first_model = list(base_model_preds.keys())[0]
    n_samples = base_model_preds[first_model].shape[0]

    # Inicializar array para predicciones
    Y_pred = np.zeros((n_samples, horizons, cells))

    # Procesar cada horizonte
    for h in range(horizons):
        logger.info(f"Generando predicciones para horizonte {h+1}/{horizontes}")

        # Si no hay modelo para este horizonte, continuar al siguiente
        if h >= len(meta_models):
            logger.warning(f"No hay modelo meta-XGB para horizonte {h+1}")
            continue

        # Preparar características para este horizonte
        X_meta_batches = []
        batch_size = 100

        # Procesar por lotes para evitar problemas de memoria
        for start_idx in range(0, n_samples, batch_size):
            end_idx = min(start_idx + batch_size, n_samples)

            # Preparar entradas para metamodelo
            X_meta_batch_parts = []
            for model_name in base_model_preds:
                if h < base_model_preds[model_name].shape[1]:
                    # Extraer predicciones del modelo base para este horizonte
                    model_preds = base_model_preds[model_name][start_idx:end_idx, h, :]
                    X_meta_batch_parts.append(model_preds.reshape(end_idx - start_idx, -1))

            # Concatenar características de todos los modelos base
            if X_meta_batch_parts:
                X_meta_batch = np.hstack(X_meta_batch_parts)

                # Predecir con el meta-modelo XGB para este lote
                Y_pred[start_idx:end_idx, h, :] = meta_models[h].predict(X_meta_batch).reshape(-1, cells)

            # Liberar memoria cada 5 lotes
            if (start_idx // batch_size) % 5 == 0 and start_idx > 0:
                # Liberar memoria explícitamente
                if 'gc' in sys.modules:
                    import gc
                    gc.collect()

    return Y_pred

# -----------------------------------------------------------------------------
# 1) Carga de datos con separación explícita de características CEEMDAN y TFV-EMD
# -----------------------------------------------------------------------------
logger.info("Cargando datasets y separando características CEEMDAN y TFV-EMD...")
ds_full = xr.open_dataset(FULL_NC)
ds_fuse = xr.open_dataset(FUSION_NC)

# precipitacion y variables
prec = ds_full["total_precipitation"].values  # (T, ny, nx)
lags = sorted([v for v in ds_full.data_vars if "_lag" in v])
da_lags = np.stack([ds_full[lag].values for lag in lags], axis=-1)  # (T, ny, nx, n_lags)

# Separar características CEEMDAN y TFV-EMD para optimizar su fusión
ceemdan_branches = ["CEEMDAN_high", "CEEMDAN_medium", "CEEMDAN_low"]
tvfemd_branches = ["TVFEMD_high", "TVFEMD_medium", "TVFEMD_low"]
fusion_branches = ["FUSION_high", "FUSION_medium", "FUSION_low"]

# Cargar datos CEEMDAN
da_ceemdan = np.stack([ds_fuse[branch].values for branch in ceemdan_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar datos TFV-EMD
da_tvfemd = np.stack([ds_fuse[branch].values for branch in tvfemd_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar fusión predefinida (para referencia)
da_fusion = np.stack([ds_fuse[branch].values for branch in fusion_branches], axis=-1)  # (T, ny, nx, 3)

# topografía y cluster
elev = ds_full["elevation"].values.ravel()  # (cells,)
slope = ds_full["slope"].values.ravel()

# Manejar correctamente los valores de cluster (pueden ser texto)
cluster_values = ds_full["cluster_elevation"].values.ravel()
# Verificar si los valores son strings o numéricos
if isinstance(cluster_values[0], (str, np.str_)):
    # Usar un LabelEncoder para convertir strings a enteros
    le = LabelEncoder()
    cluster = le.fit_transform(cluster_values)
    logger.info(f"Clusters codificados de texto a números: {dict(zip(le.classes_, range(len(le.classes_))))}")
else:
    # Si ya son numéricos, convertir a enteros
    cluster = cluster_values.astype(int)

# dimensiones
lat = ds_full.latitude.values
lon = ds_full.longitude.values
ny, nx = len(lat), len(lon)
cells = ny*nx
T = prec.shape[0]

logger.info(f"Dimensiones: T={T}, ny={ny}, nx={nx}, cells={cells}")
logger.info(f"Shapes: prec={prec.shape}, da_ceemdan={da_ceemdan.shape}, da_tvfemd={da_tvfemd.shape}")

# -----------------------------------------------------------------------------
# 2) Definir máscaras para los niveles de elevación
# -----------------------------------------------------------------------------
logger.info("Definiendo máscaras para los niveles de elevación...")
mask_nivel1 = elev < 957  # nivel_1: 58-956m
mask_nivel2 = (elev >= 957) & (elev <= 2264)  # nivel_2: 957-2264m
mask_nivel3 = elev > 2264  # nivel_3: 2264-4728m

logger.info(f"Distribución de celdas por nivel de elevación:")
logger.info(f"  Nivel 1 (<957m): {np.sum(mask_nivel1)} celdas")
logger.info(f"  Nivel 2 (957-2264m): {np.sum(mask_nivel2)} celdas")
logger.info(f"  Nivel 3 (>2264m): {np.sum(mask_nivel3)} celdas")

# Crear diccionario de máscaras para facilitar el procesamiento
elevation_masks = {
    "nivel_1": mask_nivel1,
    "nivel_2": mask_nivel2,
    "nivel_3": mask_nivel3
}

# -----------------------------------------------------------------------------
# 3) Implementar función para optimizar fusión de CEEMDAN y TFV-EMD con XGBoost
# -----------------------------------------------------------------------------
import concurrent.futures
import tqdm
from functools import partial

@trace("Optimización de fusión")
def optimize_fusion_with_xgboost(ceemdan_data, tvfemd_data, target_data, masks, test_size=0.2, force_retrain=False):
    """
    Optimiza la fusión de CEEMDAN y TFV-EMD usando XGBoost para cada nivel de elevación.
    Implementa paralelismo adaptativo basado en CPU/GPU y memoria disponible.
    
    Args:
        ceemdan_data: Array de características CEEMDAN (T, ny, nx, 3)
        tvfemd_data: Array de características TFV-EMD (T, ny, nx, 3)
        target_data: Array de valores objetivo (precipitación) (T, ny, nx)
        masks: Diccionario de máscaras por nivel de elevación
        test_size: Proporción del conjunto de prueba
        force_retrain: Si es True, fuerza el reentrenamiento aunque existan modelos guardados
        
    Returns:
        Dictionary con modelos XGBoost para fusión por nivel y componente
    """
    fusion_models = {}
    fusion_weights = {}
    
    # Comprobar si todos los modelos ya existen
    all_models_exist = True
    if not force_retrain:
        for level_name in masks:
            for component_idx in range(3):
                if not model_exists('fusion', level_name, component_idx):
                    all_models_exist = False
                    break
            if not all_models_exist:
                break
                
        if all_models_exist:
            logger.info("Todos los modelos de fusión existen. Cargando...")
            return load_all_fusion_models(masks)
    
    # Determinar recursos computacionales disponibles
    mem_info = get_memory_info()
    cpu_count = os.cpu_count()
    
    print(f"\n🖥️  Recursos detectados: {cpu_count} CPUs, {mem_info['total_gb']:.1f}GB RAM ({mem_info['available_gb']:.1f}GB disponible)")
    
    # SOLUCIÓN: Aumentar agresivamente el número de trabajadores para forzar paralelismo
    # y aprovechar mejor los recursos subutilizados
    optimal_workers = max(3, min(cpu_count - 1, 8))  # Mínimo 3 workers, máximo CPU-1 o 8
    
    # Verificar disponibilidad de GPU para tree_method
    gpu_available = len(gpus) > 0
    tree_method = 'gpu_hist' if gpu_available else 'hist'
    
    print(f"🔧 Configuración optimizada: {optimal_workers} workers en paralelo FORZADOS, tree_method={tree_method}")
    print(f"🧠 Memoria disponible: {mem_info['available_gb']:.2f}GB ({mem_info['used_percent']:.1f}% usado)")
    
    # Total de componentes a procesar
    total_levels = len(masks)
    total_components = total_levels * 3  # 3 componentes por nivel
    
    # Inicializar estructuras de datos para resultados
    for level_name in masks.keys():
        fusion_models[level_name] = [None, None, None]  # Placeholder para los 3 componentes
        fusion_weights[level_name] = [None, None, None]
    
    # Barra de progreso global
    print(f"\n📊 Iniciando entrenamiento acelerado de {total_components} componentes ({total_levels} niveles × 3 componentes)")
    
    # Función para procesar un componente específico
    def process_component(level_name, mask, component_idx):
        # Verificar si el modelo ya existe (a menos que se fuerce reentrenamiento)
        if not force_retrain and model_exists('fusion', level_name, component_idx):
            print(f"🔄 Nivel {level_name}, componente {component_idx}: Cargando modelo existente...")
            model, info = load_model('fusion', level_name, component_idx)
            if model and info:
                weights = info.get('weights')
                rmse = info.get('rmse', 0.0)
                fit_time = info.get('fit_time', 0.0)
                total_time = 0.1  # Tiempo mínimo para evitar divisiones por cero
                print(f"✅ {level_name}, comp{component_idx} (cargado): RMSE={rmse:.4f}, pesos=[CEEMDAN={weights[0]:.2f}, TFV-EMD={weights[1]:.2f}]")
                return {
                    'level': level_name,
                    'component': component_idx,
                    'model': model,
                    'weights': weights,
                    'rmse': rmse,
                    'fit_time': fit_time,
                    'total_time': total_time,
                    'loaded': True
                }
        
        # Si llegamos aquí, necesitamos entrenar el modelo
        print(f"▶️  Nivel {level_name}, componente {component_idx}: Iniciando entrenamiento rápido...")
        comp_start = time.time()
        cells_in_level = np.sum(mask)
        
        # Reformatear los datos para el entrenamiento
        X_ceemdan = ceemdan_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        X_tvfemd = tvfemd_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        y_target = target_data.reshape(T, -1)[:, mask]
        
        print(f"   Datos: {X_ceemdan.shape[0]} muestras, {cells_in_level} celdas")
        
        # Concatenar características
        X_combined = np.column_stack([X_ceemdan, X_tvfemd])
        
        # División simple para mayor velocidad (sin estratificación que consume tiempo)
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_target, test_size=test_size, random_state=42
        )
        split_method = "simple (optimizado para velocidad)"
        
        print(f"   Split: {X_train.shape[0]} train, {X_test.shape[0]} test ({split_method})")
        
        # SOLUCIÓN: Optimizar hiperparámetros para mayor velocidad
        n_samples, n_features = X_train.shape
        # Reducir profundidad y número de árboles para entrenamientos más rápidos
        max_depth = min(4, max(3, int(np.log2(n_features/2))))  # Profundidad reducida
        n_estimators = min(60, max(30, int(30 + 5 * np.log(n_samples))))  # Menos árboles
        learning_rate = min(0.3, max(0.08, 0.2))  # Learning rate más alto para convergencia rápida
        subsample = 0.7  # Usar menos datos por árbol
        colsample = 0.7  # Usar menos columnas por árbol
        
        # Configurar modelo XGBoost con paralelismo más eficiente
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample,
            tree_method=tree_method,
            n_jobs=1,  # Un hilo por modelo para maximizar paralelismo entre modelos
            enable_categorical=False,
            verbosity=0
        )
        
        # Entrenar modelo con mensaje de progreso
        print(f"   Entrenamiento ultra-rápido: {n_estimators} estimators, depth={max_depth}, lr={learning_rate:.3f}")
        fit_start = time.time()
        
        # Entrenamiento simplificado para mayor velocidad
        model.fit(
            X_train, y_train.ravel(),
            eval_set=[(X_test, y_test.ravel())],
            verbose=False
        )
        
        fit_time = time.time() - fit_start
        
        # Evaluar modelo
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test.ravel(), y_pred))
        
        # Extraer pesos de importancia para CEEMDAN vs TFV-EMD
        importance = model.feature_importances_
        cells_per_feature = cells_in_level
        
        # Promedio de importancia para cada fuente
        ceemdan_importance = np.mean(importance[:cells_per_feature])
        tvfemd_importance = np.mean(importance[cells_per_feature:])
        
        # Normalizar para que sumen 1
        total_importance = ceemdan_importance + tvfemd_importance
        ceemdan_weight = ceemdan_importance / total_importance
        tvfemd_weight = tvfemd_importance / total_importance
        
        comp_time = time.time() - comp_start
        
        print(f"✅ {level_name}, comp{component_idx}: RMSE={rmse:.4f}, tiempo={comp_time:.1f}s, "
              f"pesos=[CEEMDAN={ceemdan_weight:.2f}, TFV-EMD={tvfemd_weight:.2f}]")
        
        weights = (ceemdan_weight, tvfemd_weight)
        
        # Guardar modelo para uso futuro con información adicional
        info = {
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'training_date': datetime.datetime.now().strftime(timestamp_format),
            'hyper_params': {
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'subsample': subsample,
                'colsample_bytree': colsample
            }
        }
        
        save_model(model, 'fusion', level_name, component_idx, info)
        
        # Devolver resultados
        return {
            'level': level_name,
            'component': component_idx,
            'model': model,
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'loaded': False
        }
    
    # Procesar niveles y componentes usando paralelismo adaptativo
    all_tasks = []
    for level_name, mask in masks.items():
        # Crear tareas para todos los componentes
        for component_idx in range(3):
            all_tasks.append((level_name, mask, component_idx))
    
    # SOLUCIÓN: FORZAR paralelismo siempre
    all_results = []
    
    # Mostrar mensaje claro sobre el modo paralelo
    print(f"\n⚡ Activando procesamiento paralelo forzado con {optimal_workers} workers para acelerar el entrenamiento")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=optimal_workers) as executor:
        # Crear lista de futuros
        futures = []
        for level_name, mask, component_idx in all_tasks:
            futures.append(executor.submit(
                process_component, level_name, mask, component_idx
            ))
        
        # Mostrar progreso mientras se completan las tareas
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            progress = completed / len(futures)
            print(f"⏳ Progreso global: {completed}/{len(futures)} componentes ({progress:.1%})")
            
            try:
                result = future.result()
                all_results.append(result)
            except Exception as e:
                logger.error(f"Error en componente: {str(e)}")
    
    # Organizar resultados por nivel y componente
    for result in all_results:
        level = result['level']
        component = result['component']
        
        # Guardar modelo y pesos
        fusion_models[level][component] = result['model']
        fusion_weights[level][component] = result['weights']
        
        # Registrar métricas para trazabilidad
        tracker.log_metric(f"{level}_comp{component}", "rmse", result['rmse'])
        tracker.log_metric(f"{level}_comp{component}", "ceemdan_weight", result['weights'][0])
        tracker.log_metric(f"{level}_comp{component}", "tvfemd_weight", result['weights'][1])
        tracker.log_metric(f"{level}_comp{component}", "train_time", result['fit_time'])
        tracker.log_metric(f"{level}_comp{component}", "total_time", result['total_time'])
        tracker.log_metric(f"{level}_comp{component}", "loaded", result.get('loaded', False))
    
    # Resumen final
    print("\n🏁 Optimización de fusión completada:")
    for level_name, components in fusion_models.items():
        valid_components = sum(1 for model in components if model is not None)
        print(f"  - {level_name}: {valid_components}/3 componentes entrenados")
    
    tracker.add_checkpoint("Optimización de fusión completada", 
                          {"num_models": sum(len(models) for models in fusion_models.values())})
    
    return fusion_models, fusion_weights

def load_all_fusion_models(masks):
    """
    Carga todos los modelos de fusión existentes
    
    Args:
        masks: Diccionario de máscaras por nivel de elevación
        
    Returns:
        tuple: (fusion_models, fusion_weights)
    """
    fusion_models = {}
    fusion_weights = {}
    
    for level_name in masks.keys():
        fusion_models[level_name] = [None, None, None]
        fusion_weights[level_name] = [None, None, None]
        
        # Cargar los tres modelos de componentes
        for component_idx in range(3):
            model, info = load_model('fusion', level_name, component_idx)
            
            if model is not None and info is not None:
                fusion_models[level_name][component_idx] = model
                fusion_weights[level_name][component_idx] = info['weights']
                logger.info(f"Modelo fusión {level_name}, componente {component_idx} cargado")
                
                # Registrar métricas para trazabilidad
                tracker.log_metric(f"{level_name}_comp{component_idx}", "rmse", info.get('rmse', 0))
                tracker.log_metric(f"{level_name}_comp{component_idx}", "ceemdan_weight", info['weights'][0])
                tracker.log_metric(f"{level_name}_comp{component_idx}", "tvfemd_weight", info['weights'][1])
                tracker.log_metric(f"{level_name}_comp{component_idx}", "loaded", True)
            else:
                logger.warning(f"No se pudo cargar el modelo fusión {level_name}, componente {component_idx}")
    
    return fusion_models, fusion_weights

# -----------------------------------------------------------------------------
# 1) Carga de datos con separación explícita de características CEEMDAN y TFV-EMD
# -----------------------------------------------------------------------------
logger.info("Cargando datasets y separando características CEEMDAN y TFV-EMD...")
ds_full = xr.open_dataset(FULL_NC)
ds_fuse = xr.open_dataset(FUSION_NC)

# precipitacion y variables
prec = ds_full["total_precipitation"].values  # (T, ny, nx)
lags = sorted([v for v in ds_full.data_vars if "_lag" in v])
da_lags = np.stack([ds_full[lag].values for lag in lags], axis=-1)  # (T, ny, nx, n_lags)

# Separar características CEEMDAN y TFV-EMD para optimizar su fusión
ceemdan_branches = ["CEEMDAN_high", "CEEMDAN_medium", "CEEMDAN_low"]
tvfemd_branches = ["TVFEMD_high", "TVFEMD_medium", "TVFEMD_low"]
fusion_branches = ["FUSION_high", "FUSION_medium", "FUSION_low"]

# Cargar datos CEEMDAN
da_ceemdan = np.stack([ds_fuse[branch].values for branch in ceemdan_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar datos TFV-EMD
da_tvfemd = np.stack([ds_fuse[branch].values for branch in tvfemd_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar fusión predefinida (para referencia)
da_fusion = np.stack([ds_fuse[branch].values for branch in fusion_branches], axis=-1)  # (T, ny, nx, 3)

# topografía y cluster
elev = ds_full["elevation"].values.ravel()  # (cells,)
slope = ds_full["slope"].values.ravel()

# Manejar correctamente los valores de cluster (pueden ser texto)
cluster_values = ds_full["cluster_elevation"].values.ravel()
# Verificar si los valores son strings o numéricos
if isinstance(cluster_values[0], (str, np.str_)):
    # Usar un LabelEncoder para convertir strings a enteros
    le = LabelEncoder()
    cluster = le.fit_transform(cluster_values)
    logger.info(f"Clusters codificados de texto a números: {dict(zip(le.classes_, range(len(le.classes_))))}")
else:
    # Si ya son numéricos, convertir a enteros
    cluster = cluster_values.astype(int)

# dimensiones
lat = ds_full.latitude.values
lon = ds_full.longitude.values
ny, nx = len(lat), len(lon)
cells = ny*nx
T = prec.shape[0]

logger.info(f"Dimensiones: T={T}, ny={ny}, nx={nx}, cells={cells}")
logger.info(f"Shapes: prec={prec.shape}, da_ceemdan={da_ceemdan.shape}, da_tvfemd={da_tvfemd.shape}")

# -----------------------------------------------------------------------------
# 2) Definir máscaras para los niveles de elevación
# -----------------------------------------------------------------------------
logger.info("Definiendo máscaras para los niveles de elevación...")
mask_nivel1 = elev < 957  # nivel_1: 58-956m
mask_nivel2 = (elev >= 957) & (elev <= 2264)  # nivel_2: 957-2264m
mask_nivel3 = elev > 2264  # nivel_3: 2264-4728m

logger.info(f"Distribución de celdas por nivel de elevación:")
logger.info(f"  Nivel 1 (<957m): {np.sum(mask_nivel1)} celdas")
logger.info(f"  Nivel 2 (957-2264m): {np.sum(mask_nivel2)} celdas")
logger.info(f"  Nivel 3 (>2264m): {np.sum(mask_nivel3)} celdas")

# Crear diccionario de máscaras para facilitar el procesamiento
elevation_masks = {
    "nivel_1": mask_nivel1,
    "nivel_2": mask_nivel2,
    "nivel_3": mask_nivel3
}

# -----------------------------------------------------------------------------
# 3) Implementar función para optimizar fusión de CEEMDAN y TFV-EMD con XGBoost
# -----------------------------------------------------------------------------
import concurrent.futures
import tqdm
from functools import partial

@trace("Optimización de fusión")
def optimize_fusion_with_xgboost(ceemdan_data, tvfemd_data, target_data, masks, test_size=0.2, force_retrain=False):
    """
    Optimiza la fusión de CEEMDAN y TFV-EMD usando XGBoost para cada nivel de elevación.
    Implementa paralelismo adaptativo basado en CPU/GPU y memoria disponible.
    
    Args:
        ceemdan_data: Array de características CEEMDAN (T, ny, nx, 3)
        tvfemd_data: Array de características TFV-EMD (T, ny, nx, 3)
        target_data: Array de valores objetivo (precipitación) (T, ny, nx)
        masks: Diccionario de máscaras por nivel de elevación
        test_size: Proporción del conjunto de prueba
        force_retrain: Si es True, fuerza el reentrenamiento aunque existan modelos guardados
        
    Returns:
        Dictionary con modelos XGBoost para fusión por nivel y componente
    """
    fusion_models = {}
    fusion_weights = {}
    
    # Comprobar si todos los modelos ya existen
    all_models_exist = True
    if not force_retrain:
        for level_name in masks:
            for component_idx in range(3):
                if not model_exists('fusion', level_name, component_idx):
                    all_models_exist = False
                    break
            if not all_models_exist:
                break
                
        if all_models_exist:
            logger.info("Todos los modelos de fusión existen. Cargando...")
            return load_all_fusion_models(masks)
    
    # Determinar recursos computacionales disponibles
    mem_info = get_memory_info()
    cpu_count = os.cpu_count()
    
    print(f"\n🖥️  Recursos detectados: {cpu_count} CPUs, {mem_info['total_gb']:.1f}GB RAM ({mem_info['available_gb']:.1f}GB disponible)")
    
    # SOLUCIÓN: Aumentar agresivamente el número de trabajadores para forzar paralelismo
    # y aprovechar mejor los recursos subutilizados
    optimal_workers = max(3, min(cpu_count - 1, 8))  # Mínimo 3 workers, máximo CPU-1 o 8
    
    # Verificar disponibilidad de GPU para tree_method
    gpu_available = len(gpus) > 0
    tree_method = 'gpu_hist' if gpu_available else 'hist'
    
    print(f"🔧 Configuración optimizada: {optimal_workers} workers en paralelo FORZADOS, tree_method={tree_method}")
    print(f"🧠 Memoria disponible: {mem_info['available_gb']:.2f}GB ({mem_info['used_percent']:.1f}% usado)")
    
    # Total de componentes a procesar
    total_levels = len(masks)
    total_components = total_levels * 3  # 3 componentes por nivel
    
    # Inicializar estructuras de datos para resultados
    for level_name in masks.keys():
        fusion_models[level_name] = [None, None, None]  # Placeholder para los 3 componentes
        fusion_weights[level_name] = [None, None, None]
    
    # Barra de progreso global
    print(f"\n📊 Iniciando entrenamiento acelerado de {total_components} componentes ({total_levels} niveles × 3 componentes)")
    
    # Función para procesar un componente específico
    def process_component(level_name, mask, component_idx):
        # Verificar si el modelo ya existe (a menos que se fuerce reentrenamiento)
        if not force_retrain and model_exists('fusion', level_name, component_idx):
            print(f"🔄 Nivel {level_name}, componente {component_idx}: Cargando modelo existente...")
            model, info = load_model('fusion', level_name, component_idx)
            if model and info:
                weights = info.get('weights')
                rmse = info.get('rmse', 0.0)
                fit_time = info.get('fit_time', 0.0)
                total_time = 0.1  # Tiempo mínimo para evitar divisiones por cero
                print(f"✅ {level_name}, comp{component_idx} (cargado): RMSE={rmse:.4f}, pesos=[CEEMDAN={weights[0]:.2f}, TFV-EMD={weights[1]:.2f}]")
                return {
                    'level': level_name,
                    'component': component_idx,
                    'model': model,
                    'weights': weights,
                    'rmse': rmse,
                    'fit_time': fit_time,
                    'total_time': total_time,
                    'loaded': True
                }
        
        # Si llegamos aquí, necesitamos entrenar el modelo
        print(f"▶️  Nivel {level_name}, componente {component_idx}: Iniciando entrenamiento rápido...")
        comp_start = time.time()
        cells_in_level = np.sum(mask)
        
        # Reformatear los datos para el entrenamiento
        X_ceemdan = ceemdan_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        X_tvfemd = tvfemd_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        y_target = target_data.reshape(T, -1)[:, mask]
        
        print(f"   Datos: {X_ceemdan.shape[0]} muestras, {cells_in_level} celdas")
        
        # Concatenar características
        X_combined = np.column_stack([X_ceemdan, X_tvfemd])
        
        # División simple para mayor velocidad (sin estratificación que consume tiempo)
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_target, test_size=test_size, random_state=42
        )
        split_method = "simple (optimizado para velocidad)"
        
        print(f"   Split: {X_train.shape[0]} train, {X_test.shape[0]} test ({split_method})")
        
        # SOLUCIÓN: Optimizar hiperparámetros para mayor velocidad
        n_samples, n_features = X_train.shape
        # Reducir profundidad y número de árboles para entrenamientos más rápidos
        max_depth = min(4, max(3, int(np.log2(n_features/2))))  # Profundidad reducida
        n_estimators = min(60, max(30, int(30 + 5 * np.log(n_samples))))  # Menos árboles
        learning_rate = min(0.3, max(0.08, 0.2))  # Learning rate más alto para convergencia rápida
        subsample = 0.7  # Usar menos datos por árbol
        colsample = 0.7  # Usar menos columnas por árbol
        
        # Configurar modelo XGBoost con paralelismo más eficiente
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample,
            tree_method=tree_method,
            n_jobs=1,  # Un hilo por modelo para maximizar paralelismo entre modelos
            enable_categorical=False,
            verbosity=0
        )
        
        # Entrenar modelo con mensaje de progreso
        print(f"   Entrenamiento ultra-rápido: {n_estimators} estimators, depth={max_depth}, lr={learning_rate:.3f}")
        fit_start = time.time()
        
        # Entrenamiento simplificado para mayor velocidad
        model.fit(
            X_train, y_train.ravel(),
            eval_set=[(X_test, y_test.ravel())],
            verbose=False
        )
        
        fit_time = time.time() - fit_start
        
        # Evaluar modelo
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test.ravel(), y_pred))
        
        # Extraer pesos de importancia para CEEMDAN vs TFV-EMD
        importance = model.feature_importances_
        cells_per_feature = cells_in_level
        
        # Promedio de importancia para cada fuente
        ceemdan_importance = np.mean(importance[:cells_per_feature])
        tvfemd_importance = np.mean(importance[cells_per_feature:])
        
        # Normalizar para que sumen 1
        total_importance = ceemdan_importance + tvfemd_importance
        ceemdan_weight = ceemdan_importance / total_importance
        tvfemd_weight = tvfemd_importance / total_importance
        
        comp_time = time.time() - comp_start
        
        print(f"✅ {level_name}, comp{component_idx}: RMSE={rmse:.4f}, tiempo={comp_time:.1f}s, "
              f"pesos=[CEEMDAN={ceemdan_weight:.2f}, TFV-EMD={tvfemd_weight:.2f}]")
        
        weights = (ceemdan_weight, tvfemd_weight)
        
        # Guardar modelo para uso futuro con información adicional
        info = {
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'training_date': datetime.datetime.now().strftime(timestamp_format),
            'hyper_params': {
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'subsample': subsample,
                'colsample_bytree': colsample
            }
        }
        
        save_model(model, 'fusion', level_name, component_idx, info)
        
        # Devolver resultados
        return {
            'level': level_name,
            'component': component_idx,
            'model': model,
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'loaded': False
        }
    
    # Procesar niveles y componentes usando paralelismo adaptativo
    all_tasks = []
    for level_name, mask in masks.items():
        # Crear tareas para todos los componentes
        for component_idx in range(3):
            all_tasks.append((level_name, mask, component_idx))
    
    # SOLUCIÓN: FORZAR paralelismo siempre
    all_results = []
    
    # Mostrar mensaje claro sobre el modo paralelo
    print(f"\n⚡ Activando procesamiento paralelo forzado con {optimal_workers} workers para acelerar el entrenamiento")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=optimal_workers) as executor:
        # Crear lista de futuros
        futures = []
        for level_name, mask, component_idx in all_tasks:
            futures.append(executor.submit(
                process_component, level_name, mask, component_idx
            ))
        
        # Mostrar progreso mientras se completan las tareas
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            progress = completed / len(futures)
            print(f"⏳ Progreso global: {completed}/{len(futures)} componentes ({progress:.1%})")
            
            try:
                result = future.result()
                all_results.append(result)
            except Exception as e:
                logger.error(f"Error en componente: {str(e)}")
    
    # Organizar resultados por nivel y componente
    for result in all_results:
        level = result['level']
        component = result['component']
        
        # Guardar modelo y pesos
        fusion_models[level][component] = result['model']
        fusion_weights[level][component] = result['weights']
        
        # Registrar métricas para trazabilidad
        tracker.log_metric(f"{level}_comp{component}", "rmse", result['rmse'])
        tracker.log_metric(f"{level}_comp{component}", "ceemdan_weight", result['weights'][0])
        tracker.log_metric(f"{level}_comp{component}", "tvfemd_weight", result['weights'][1])
        tracker.log_metric(f"{level}_comp{component}", "train_time", result['fit_time'])
        tracker.log_metric(f"{level}_comp{component}", "total_time", result['total_time'])
        tracker.log_metric(f"{level}_comp{component}", "loaded", result.get('loaded', False))
    
    # Resumen final
    print("\n🏁 Optimización de fusión completada:")
    for level_name, components in fusion_models.items():
        valid_components = sum(1 for model in components if model is not None)
        print(f"  - {level_name}: {valid_components}/3 componentes entrenados")
    
    tracker.add_checkpoint("Optimización de fusión completada", 
                          {"num_models": sum(len(models) for models in fusion_models.values())})
    
    return fusion_models, fusion_weights

def load_all_fusion_models(masks):
    """
    Carga todos los modelos de fusión existentes
    
    Args:
        masks: Diccionario de máscaras por nivel de elevación
        
    Returns:
        tuple: (fusion_models, fusion_weights)
    """
    fusion_models = {}
    fusion_weights = {}
    
    for level_name in masks.keys():
        fusion_models[level_name] = [None, None, None]
        fusion_weights[level_name] = [None, None, None]
        
        # Cargar los tres modelos de componentes
        for component_idx in range(3):
            model, info = load_model('fusion', level_name, component_idx)
            
            if model is not None and info is not None:
                fusion_models[level_name][component_idx] = model
               
                fusion_weights[level_name][component_idx] = info['weights']
                logger.info(f"Modelo fusión {level_name}, componente {component_idx} cargado")
                
                # Registrar métricas para trazabilidad
                tracker.log_metric(f"{level_name}_comp{component_idx}", "rmse", info.get('rmse', 0))
                tracker.log_metric(f"{level_name}_comp{component_idx}", "ceemdan_weight", info['weights'][0])
                tracker.log_metric(f"{level_name}_comp{component_idx}", "tvfemd_weight", info['weights'][1])
                tracker.log_metric(f"{level_name}_comp{component_idx}", "loaded", True)
            else:
                logger.warning(f"No se pudo cargar el modelo fusión {level_name}, componente {component_idx}")
    
    return fusion_models, fusion_weights

# -----------------------------------------------------------------------------
# 1) Carga de datos con separación explícita de características CEEMDAN y TFV-EMD
# -----------------------------------------------------------------------------
logger.info("Cargando datasets y separando características CEEMDAN y TFV-EMD...")
ds_full = xr.open_dataset(FULL_NC)
ds_fuse = xr.open_dataset(FUSION_NC)

# precipitacion y variables
prec = ds_full["total_precipitation"].values  # (T, ny, nx)
lags = sorted([v for v in ds_full.data_vars if "_lag" in v])
da_lags = np.stack([ds_full[lag].values for lag in lags], axis=-1)  # (T, ny, nx, n_lags)

# Separar características CEEMDAN y TFV-EMD para optimizar su fusión
ceemdan_branches = ["CEEMDAN_high", "CEEMDAN_medium", "CEEMDAN_low"]
tvfemd_branches = ["TVFEMD_high", "TVFEMD_medium", "TVFEMD_low"]
fusion_branches = ["FUSION_high", "FUSION_medium", "FUSION_low"]

# Cargar datos CEEMDAN
da_ceemdan = np.stack([ds_fuse[branch].values for branch in ceemdan_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar datos TFV-EMD
da_tvfemd = np.stack([ds_fuse[branch].values for branch in tvfemd_branches], axis=-1)  # (T, ny, nx, 3)
# Cargar fusión predefinida (para referencia)
da_fusion = np.stack([ds_fuse[branch].values for branch in fusion_branches], axis=-1)  # (T, ny, nx, 3)

# topografía y cluster
elev = ds_full["elevation"].values.ravel()  # (cells,)
slope = ds_full["slope"].values.ravel()

# Manejar correctamente los valores de cluster (pueden ser texto)
cluster_values = ds_full["cluster_elevation"].values.ravel()
# Verificar si los valores son strings o numéricos
if isinstance(cluster_values[0], (str, np.str_)):
    # Usar un LabelEncoder para convertir strings a enteros
    le = LabelEncoder()
    cluster = le.fit_transform(cluster_values)
    logger.info(f"Clusters codificados de texto a números: {dict(zip(le.classes_, range(len(le.classes_))))}")
else:
    # Si ya son numéricos, convertir a enteros
    cluster = cluster_values.astype(int)

# dimensiones
lat = ds_full.latitude.values
lon = ds_full.longitude.values
ny, nx = len(lat), len(lon)
cells = ny*nx
T = prec.shape[0]

logger.info(f"Dimensiones: T={T}, ny={ny}, nx={nx}, cells={cells}")
logger.info(f"Shapes: prec={prec.shape}, da_ceemdan={da_ceemdan.shape}, da_tvfemd={da_tvfemd.shape}")

# -----------------------------------------------------------------------------
# 2) Definir máscaras para los niveles de elevación
# -----------------------------------------------------------------------------
logger.info("Definiendo máscaras para los niveles de elevación...")
mask_nivel1 = elev < 957  # nivel_1: 58-956m
mask_nivel2 = (elev >= 957) & (elev <= 2264)  # nivel_2: 957-2264m
mask_nivel3 = elev > 2264  # nivel_3: 2264-4728m

logger.info(f"Distribución de celdas por nivel de elevación:")
logger.info(f"  Nivel 1 (<957m): {np.sum(mask_nivel1)} celdas")
logger.info(f"  Nivel 2 (957-2264m): {np.sum(mask_nivel2)} celdas")
logger.info(f"  Nivel 3 (>2264m): {np.sum(mask_nivel3)} celdas")

# Crear diccionario de máscaras para facilitar el procesamiento
elevation_masks = {
    "nivel_1": mask_nivel1,
    "nivel_2": mask_nivel2,
    "nivel_3": mask_nivel3
}

# -----------------------------------------------------------------------------
# 3) Implementar función para optimizar fusión de CEEMDAN y TFV-EMD con XGBoost
# -----------------------------------------------------------------------------
import concurrent.futures
import tqdm
from functools import partial

@trace("Optimización de fusión")
def optimize_fusion_with_xgboost(ceemdan_data, tvfemd_data, target_data, masks, test_size=0.2, force_retrain=False):
    """
    Optimiza la fusión de CEEMDAN y TFV-EMD usando XGBoost para cada nivel de elevación.
    Implementa paralelismo adaptativo basado en CPU/GPU y memoria disponible.
    
    Args:
        ceemdan_data: Array de características CEEMDAN (T, ny, nx, 3)
        tvfemd_data: Array de características TFV-EMD (T, ny, nx, 3)
        target_data: Array de valores objetivo (precipitación) (T, ny, nx)
        masks: Diccionario de máscaras por nivel de elevación
        test_size: Proporción del conjunto de prueba
        force_retrain: Si es True, fuerza el reentrenamiento aunque existan modelos guardados
        
    Returns:
        Dictionary con modelos XGBoost para fusión por nivel y componente
    """
    fusion_models = {}
    fusion_weights = {}
    
    # Comprobar si todos los modelos ya existen
    all_models_exist = True
    if not force_retrain:
        for level_name in masks:
            for component_idx in range(3):
                if not model_exists('fusion', level_name, component_idx):
                    all_models_exist = False
                    break
            if not all_models_exist:
                break
                
        if all_models_exist:
            logger.info("Todos los modelos de fusión existen. Cargando...")
            return load_all_fusion_models(masks)
    
    # Determinar recursos computacionales disponibles
    mem_info = get_memory_info()
    cpu_count = os.cpu_count()
    
    print(f"\n🖥️  Recursos detectados: {cpu_count} CPUs, {mem_info['total_gb']:.1f}GB RAM ({mem_info['available_gb']:.1f}GB disponible)")
    
    # SOLUCIÓN: Aumentar agresivamente el número de trabajadores para forzar paralelismo
    # y aprovechar mejor los recursos subutilizados
    optimal_workers = max(3, min(cpu_count - 1, 8))  # Mínimo 3 workers, máximo CPU-1 o 8
    
    # Verificar disponibilidad de GPU para tree_method
    gpu_available = len(gpus) > 0
    tree_method = 'gpu_hist' if gpu_available else 'hist'
    
    print(f"🔧 Configuración optimizada: {optimal_workers} workers en paralelo FORZADOS, tree_method={tree_method}")
    print(f"🧠 Memoria disponible: {mem_info['available_gb']:.2f}GB ({mem_info['used_percent']:.1f}% usado)")
    
    # Total de componentes a procesar
    total_levels = len(masks)
    total_components = total_levels * 3  # 3 componentes por nivel
    
    # Inicializar estructuras de datos para resultados
    for level_name in masks.keys():
        fusion_models[level_name] = [None, None, None]  # Placeholder para los 3 componentes
        fusion_weights[level_name] = [None, None, None]
    
    # Barra de progreso global
    print(f"\n📊 Iniciando entrenamiento acelerado de {total_components} componentes ({total_levels} niveles × 3 componentes)")
    
    # Función para procesar un componente específico
    def process_component(level_name, mask, component_idx):
        # Verificar si el modelo ya existe (a menos que se fuerce reentrenamiento)
        if not force_retrain and model_exists('fusion', level_name, component_idx):
            print(f"🔄 Nivel {level_name}, componente {component_idx}: Cargando modelo existente...")
            model, info = load_model('fusion', level_name, component_idx)
            if model and info:
                weights = info.get('weights')
                rmse = info.get('rmse', 0.0)
                fit_time = info.get('fit_time', 0.0)
                total_time = 0.1  # Tiempo mínimo para evitar divisiones por cero
                print(f"✅ {level_name}, comp{component_idx} (cargado): RMSE={rmse:.4f}, pesos=[CEEMDAN={weights[0]:.2f}, TFV-EMD={weights[1]:.2f}]")
                return {
                    'level': level_name,
                    'component': component_idx,
                    'model': model,
                    'weights': weights,
                    'rmse': rmse,
                    'fit_time': fit_time,
                    'total_time': total_time,
                    'loaded': True
                }
        
        # Si llegamos aquí, necesitamos entrenar el modelo
        print(f"▶️  Nivel {level_name}, componente {component_idx}: Iniciando entrenamiento rápido...")
        comp_start = time.time()
        cells_in_level = np.sum(mask)
        
        # Reformatear los datos para el entrenamiento
        X_ceemdan = ceemdan_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        X_tvfemd = tvfemd_data[:, :, :, component_idx].reshape(T, -1)[:, mask]
        y_target = target_data.reshape(T, -1)[:, mask]
        
        print(f"   Datos: {X_ceemdan.shape[0]} muestras, {cells_in_level} celdas")
        
        # Concatenar características
        X_combined = np.column_stack([X_ceemdan, X_tvfemd])
        
        # División simple para mayor velocidad (sin estratificación que consume tiempo)
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_target, test_size=test_size, random_state=42
        )
        split_method = "simple (optimizado para velocidad)"
        
        print(f"   Split: {X_train.shape[0]} train, {X_test.shape[0]} test ({split_method})")
        
        # SOLUCIÓN: Optimizar hiperparámetros para mayor velocidad
        n_samples, n_features = X_train.shape
        # Reducir profundidad y número de árboles para entrenamientos más rápidos
        max_depth = min(4, max(3, int(np.log2(n_features/2))))  # Profundidad reducida
        n_estimators = min(60, max(30, int(30 + 5 * np.log(n_samples))))  # Menos árboles
        learning_rate = min(0.3, max(0.08, 0.2))  # Learning rate más alto para convergencia rápida
        subsample = 0.7  # Usar menos datos por árbol
        colsample = 0.7  # Usar menos columnas por árbol
        
        # Configurar modelo XGBoost con paralelismo más eficiente
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample,
            tree_method=tree_method,
            n_jobs=1,  # Un hilo por modelo para maximizar paralelismo entre modelos
            enable_categorical=False,
            verbosity=0
        )
        
        # Entrenar modelo con mensaje de progreso
        print(f"   Entrenamiento ultra-rápido: {n_estimators} estimators, depth={max_depth}, lr={learning_rate:.3f}")
        fit_start = time.time()
        
        # Entrenamiento simplificado para mayor velocidad
        model.fit(
            X_train, y_train.ravel(),
            eval_set=[(X_test, y_test.ravel())],
            verbose=False
        )
        
        fit_time = time.time() - fit_start
        
        # Evaluar modelo
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test.ravel(), y_pred))
        
        # Extraer pesos de importancia para CEEMDAN vs TFV-EMD
        importance = model.feature_importances_
        cells_per_feature = cells_in_level
        
        # Promedio de importancia para cada fuente
        ceemdan_importance = np.mean(importance[:cells_per_feature])
        tvfemd_importance = np.mean(importance[cells_per_feature:])
        
        # Normalizar para que sumen 1
        total_importance = ceemdan_importance + tvfemd_importance
        ceemdan_weight = ceemdan_importance / total_importance
        tvfemd_weight = tvfemd_importance / total_importance
        
        comp_time = time.time() - comp_start
        
        print(f"✅ {level_name}, comp{component_idx}: RMSE={rmse:.4f}, tiempo={comp_time:.1f}s, "
              f"pesos=[CEEMDAN={ceemdan_weight:.2f}, TFV-EMD={tvfemd_weight:.2f}]")
        
        weights = (ceemdan_weight, tvfemd_weight)
        
        # Guardar modelo para uso futuro con información adicional
        info = {
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'training_date': datetime.datetime.now().strftime(timestamp_format),
            'hyper_params': {
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'learning_rate': learning_rate,
                'subsample': subsample,
                'colsample_bytree': colsample
            }
        }
        
        save_model(model, 'fusion', level_name, component_idx, info)
        
        # Devolver resultados
        return {
            'level': level_name,
            'component': component_idx,
            'model': model,
            'weights': weights,
            'rmse': rmse,
            'fit_time': fit_time,
            'total_time': comp_time,
            'loaded': False
        }
    
    # Procesar niveles y componentes usando paralelismo adaptativo
    all_tasks = []
    for level_name, mask in masks.items():
        # Crear tareas para todos los componentes
        for component_idx in range(3):
            all_tasks.append((level_name, mask, component_idx))
    
    # SOLUCIÓN: FORZAR paralelismo siempre
    all_results = []
    
    # Mostrar mensaje claro sobre el modo paralelo
    print(f"\n⚡ Activando procesamiento paralelo forzado con {optimal_workers} workers para acelerar el entrenamiento")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=optimal_workers) as executor:
        # Crear lista de futuros
        futures = []
        for level_name, mask, component_idx in all_tasks:
            futures.append(executor.submit(
                process_component, level_name, mask, component_idx
            ))
        
        # Mostrar progreso mientras se completan las tareas
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            progress = completed / len(futures)
            print(f"⏳ Progreso global: {completed}/{len(futures)} componentes ({progress:.1%})")
            
            try:
                result = future.result()
                all_results.append(result)
            except Exception as e:
                logger.error(f"Error en componente: {str(e)}")
    
    # Organizar resultados por nivel y componente
    for result in all_results:
        level = result['level']
        component = result['component']
        
        # Guardar modelo y pesos
        fusion_models[level][component] = result['model']
        fusion_weights[level][component] = result['weights']
        
        # Registrar métricas para trazabilidad
        tracker.log_metric(f"{level}_comp{component}", "rmse", result['rmse'])
        tracker.log_metric(f"{level}_comp{component}", "ceemdan_weight", result['weights'][0])
        tracker.log_metric(f"{level}_comp{component}", "tvfemd_weight", result['weights'][1])
        tracker.log_metric(f"{level}_comp{component}", "train_time", result['fit_time'])
        tracker.log_metric(f"{level}_comp{component}", "total_time", result['total_time'])
        tracker.log_metric(f"{level}_comp{component}", "loaded", result.get('loaded', False))
    
    # Resumen final
    print("\n🏁 Optimización de fusión completada:")
    for level_name, components in fusion_models.items():
        valid_components = sum(1 for model in components if model is not None)
        print(f"  - {level_name}: {valid_components}/3 componentes entrenados")
    
    tracker.add_checkpoint("Optimización de fusión completada", 
                          {"num_models": sum(len(models) for models in fusion_models.values())})
    
    return fusion_models, fusion_weights

# Funciones para evaluación de modelos
@trace("Evaluación global de modelos")
def calculate_global_metrics(predictions, ground_truth):
    """
    Calcula métricas globales para todos los modelos.
    
    Args:
        predictions: Diccionario de predicciones por modelo
        ground_truth: Valores reales de precipitación
        
    Returns:
        DataFrame con métricas para cada modelo
    """
    import pandas as pd
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    metrics_list = []
    
    logger.info("Calculando métricas globales para todos los modelos...")
    
    for model_name, preds in predictions.items():
        # Aplanar arrays para cálculo de métricas globales
        y_true = ground_truth.reshape(-1)
        y_pred = preds.reshape(-1)
        
        # Filtrar NaN si existen
        mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
        if np.sum(mask) < len(mask):
            logger.warning(f"Modelo {model_name}: {len(mask) - np.sum(mask)} valores NaN detectados y excluidos")
            y_true = y_true[mask]
            y_pred = y_pred[mask]
        
        # Calcular métricas
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        
        # MAPE con manejo de divisiones por cero
        mask_nonzero = y_true != 0
        if np.sum(mask_nonzero) > 0:
            mape = 100 * np.mean(np.abs((y_true[mask_nonzero] - y_pred[mask_nonzero]) / y_true[mask_nonzero]))
        else:
            mape = np.nan
        
        metrics_list.append({
            'Model': model_name,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'R²': r2
        })
        
        logger.info(f"Modelo {model_name}: MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%, R²={r2:.4f}")
    
    # Crear DataFrame con métricas
    metrics_df = pd.DataFrame(metrics_list)
    return metrics_df

@trace("Evaluación por niveles de elevación")
def calculate_metrics_by_elevation(predictions, ground_truth, elevation_masks):
    """
    Calcula métricas separadas por nivel de elevación para todos los modelos.
    
    Args:
        predictions: Diccionario de predicciones por modelo
        ground_truth: Valores reales de precipitación
        elevation_masks: Diccionario de máscaras por nivel de elevación
        
    Returns:
        DataFrame con métricas para cada modelo y nivel de elevación
    """
    import pandas as pd
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    metrics_list = []
    
    logger.info("Calculando métricas por nivel de elevación...")
    
    # Para cada nivel de elevación
    for level_name, mask in elevation_masks.items():
        # Máscara a índices
        level_indices = np.where(mask)[0]
        
        # Para cada modelo
        for model_name, preds in predictions.items():
            # Preparar datos para este nivel
            y_true_level = []
            y_pred_level = []
            
            # Recopilar predicciones para todos los timesteps y horizontes
            for t in range(ground_truth.shape[0]):
                for h in range(ground_truth.shape[1]):
                    y_true_level.append(ground_truth[t, h, level_indices])
                    y_pred_level.append(preds[t, h, level_indices])
            
            # Convertir a arrays y aplanar
            y_true_level = np.concatenate(y_true_level)
            y_pred_level = np.concatenate(y_pred_level)
            
            # Filtrar NaN si existen
            mask_valid = ~np.isnan(y_true_level) & ~np.isnan(y_pred_level)
            if np.sum(mask_valid) < len(mask_valid):
                logger.warning(f"Modelo {model_name}, nivel {level_name}: {len(mask_valid) - np.sum(mask_valid)} valores NaN detectados y excluidos")
                y_true_level = y_true_level[mask_valid]
                y_pred_level = y_pred_level[mask_valid]
            
            # Calcular métricas para este nivel
            mae = mean_absolute_error(y_true_level, y_pred_level)
            rmse = np.sqrt(mean_squared_error(y_true_level, y_pred_level))
            r2 = r2_score(y_true_level, y_pred_level)
            
            # MAPE con manejo de divisiones por cero
            mask_nonzero = y_true_level != 0
            if np.sum(mask_nonzero) > 0:
                mape = 100 * np.mean(np.abs((y_true_level[mask_nonzero] - y_pred_level[mask_nonzero]) / y_true_level[mask_nonzero]))
            else:
                mape = np.nan
            
            metrics_list.append({
                'Model': model_name,
                'Elevation Level': level_name,
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'R²': r2
            })
            
            logger.info(f"Modelo {model_name}, nivel {level_name}: MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%, R²={r2:.4f}")
    
    # Crear DataFrame con métricas
    metrics_df = pd.DataFrame(metrics_list)
    return metrics_df

@trace("Evaluación por percentiles")
def calculate_metrics_by_percentiles(predictions, ground_truth, percentiles):
    """
    Calcula métricas separadas por rangos de percentiles para todos los modelos.
    
    Args:
        predictions: Diccionario de predicciones por modelo
        ground_truth: Valores reales de precipitación
        percentiles: Lista de percentiles para definir los rangos
        
    Returns:
        DataFrame con métricas para cada modelo y rango de percentil
    """
    import pandas as pd
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    metrics_list = []
    
    logger.info(f"Calculando métricas por percentiles: {percentiles}")
    
    # Calcular umbrales de percentiles en los datos reales
    y_true_flat = ground_truth.reshape(-1)
    y_true_nonzero = y_true_flat[y_true_flat > 0]  # Solo valores positivos
    
    if len(y_true_nonzero) == 0:
        logger.warning("No hay valores positivos para calcular percentiles. Omitiendo cálculo por percentiles.")
        return pd.DataFrame()
    
    # Calcular umbrales
    thresholds = [np.percentile(y_true_nonzero, p) for p in percentiles]
    logger.info(f"Umbrales de percentiles: {thresholds}")
    
    # Para cada rango de percentiles
    for i in range(len(percentiles)-1):
        lower_pct = percentiles[i]
        upper_pct = percentiles[i+1]
        lower_val = thresholds[i]
        upper_val = thresholds[i+1]
        
        range_name = f"P{lower_pct}-P{upper_pct}"
        logger.info(f"Calculando métricas para rango {range_name}: {lower_val:.4f} - {upper_val:.4f}")
        
        # Para cada modelo
        for model_name, preds in predictions.items():
            # Aplanar arrays
            y_true = ground_truth.reshape(-1)
            y_pred = preds.reshape(-1)
            
            # Filtrar por rango de percentiles
            if i == len(percentiles)-2:  # Último rango, incluir el valor superior
                mask_range = (y_true >= lower_val) & (y_true <= upper_val)
            else:
                mask_range = (y_true >= lower_val) & (y_true < upper_val)
            
            # Si no hay datos en este rango, continuar
            if np.sum(mask_range) == 0:
                logger.warning(f"No hay datos en rango {range_name} para modelo {model_name}")
                continue
                
            # Filtrar datos para este rango
            y_true_range = y_true[mask_range]
            y_pred_range = y_pred[mask_range]
            
            # Filtrar NaN si existen
            mask_valid = ~np.isnan(y_true_range) & ~np.isnan(y_pred_range)
            if np.sum(mask_valid) < len(mask_valid):
                logger.warning(f"Modelo {model_name}, rango {range_name}: {len(mask_valid) - np.sum(mask_valid)} valores NaN detectados y excluidos")
                y_true_range = y_true_range[mask_valid]
                y_pred_range = y_pred_range[mask_valid]
            
            # Calcular métricas para este rango
            mae = mean_absolute_error(y_true_range, y_pred_range)
            rmse = np.sqrt(mean_squared_error(y_true_range, y_pred_range))
            r2 = r2_score(y_true_range, y_pred_range)
            
            # MAPE con manejo de divisiones por cero
            mask_nonzero = y_true_range != 0
            if np.sum(mask_nonzero) > 0:
                mape = 100 * np.mean(np.abs((y_true_range[mask_nonzero] - y_pred_range[mask_nonzero]) / y_true_range[mask_nonzero]))
            else:
                mape = np.nan
            
            metrics_list.append({
                'Model': model_name,
                'Percentile Range': range_name,
                'Value Range': f"{lower_val:.4f} - {upper_val:.4f}",
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'R²': r2,
                'Samples': np.sum(mask_range)
            })
            
            logger.info(f"Modelo {model_name}, rango {range_name}: MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.2f}%, R²={r2:.4f}")
    
    # Crear DataFrame con métricas
    metrics_df = pd.DataFrame(metrics_list)
    return metrics_df

@trace("Visualización de mapas de predicción")
def plot_all_model_maps(predictions, ground_truth, lat, lon, example_idx=0, horizon_idx=0):
    """
    Genera mapas de predicciones para todos los modelos en un horizonte específico.
    
    Args:
        predictions: Diccionario de predicciones por modelo
        ground_truth: Valores reales de precipitación
        lat: Latitudes para el mapa
        lon: Longitudes para el mapa
        example_idx: Índice del ejemplo a visualizar
        horizon_idx: Índice del horizonte a visualizar
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    # Configurar visualización
    n_models = len(predictions) + 1  # +1 para ground truth
    n_cols = min(3, n_models)
    n_rows = (n_models + n_cols - 1) // n_cols
    
    # Crear figura
    fig = plt.figure(figsize=(n_cols * 5, n_rows * 4))
    
    # Encontrar límites de colorbar consistentes para todos los mapas
    vmin = ground_truth[example_idx, horizon_idx].min()
    vmax = ground_truth[example_idx, horizon_idx].max()
    
    # Crear mapa para ground truth primero
    ax = plt.subplot(n_rows, n_cols, 1, projection=ccrs.PlateCarree())
    
    # Reshape de datos para mapeo
    ny, nx = len(lat), len(lon)
    truth_map = ground_truth[example_idx, horizon_idx].reshape(ny, nx)
    
    # Configurar colormap para precipitación
    cmap = plt.cm.YlGnBu
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    
    # Añadir características del mapa
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.RIVERS, linestyle='-', alpha=0.5)
    
    # Plotear datos
    im = ax.pcolormesh(lon, lat, truth_map, cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    
    # Añadir colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05, axes_class=plt.Axes)
    plt.colorbar(im, cax=cax, orientation="vertical", label="Precipitation")
    
    # Título y configuración
    ax.set_title(f"Ground Truth (H+{horizon_idx+1})")
    ax.gridlines(draw_labels=True, alpha=0.3)
    
    # Crear mapas para cada modelo
    for i, (model_name, preds) in enumerate(predictions.items(), 2):
        ax = plt.subplot(n_rows, n_cols, i, projection=ccrs.PlateCarree())
        
        # Reshape de datos para mapeo
        pred_map = preds[example_idx, horizon_idx].reshape(ny, nx)
        
        # Añadir características del mapa
        ax.coastlines(resolution='10m')
        ax.add_feature(cfeature.BORDERS, linestyle=':')
        ax.add_feature(cfeature.RIVERS, linestyle='-', alpha=0.5)
        
        # Plotear datos
        im = ax.pcolormesh(lon, lat, pred_map, cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
        
        # Añadir colorbar
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05, axes_class=plt.Axes)
        plt.colorbar(im, cax=cax, orientation="vertical", label="Precipitation")
        
        # Título y configuración
        ax.set_title(f"{model_name} (H+{horizon_idx+1})")
        ax.gridlines(draw_labels=True, alpha=0.3)
    
    # Ajustar diseño y guardar
    plt.tight_layout()
    plt.savefig(f"{BASE}/models/output/prediction_maps_horizon{horizonte_idx+1}.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    logger.info(f"Mapas de predicción guardados para horizonte {horizonte_idx+1}")

def visualize_process_tracker_results():
    """Visualiza los resultados del tracker de proceso"""
    import seaborn as sns
    
    # Crear directorio de salida si no existe
    vis_dir = Path(f"{BASE}/models/output/visualizations")
    vis_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. Visualizar tiempo por sección
    plt.figure(figsize=(12, 6))
    section_names = list(tracker.section_times.keys())
    section_times = list(tracker.section_times.values())
    
    # Ordenar por tiempo
    indices = np.argsort(section_times)
    section_names = [section_names[i] for i in indices]
    section_times = [section_times[i] for i in indices]
    
    sns.barplot(x=section_times, y=section_names)
    plt.title('Tiempo de ejecución por sección')
    plt.xlabel('Tiempo (segundos)')
    plt.tight_layout()
    plt.savefig(f"{vis_dir}/section_times.png", dpi=300)
    plt.close()
    
    # 2. Visualizar uso de recursos a lo largo del tiempo
    if tracker.resources:
        times = [(r['timestamp'] - tracker.start_time) for r in tracker.resources]
        mem_pcts = [r['memory_percent'] for r in tracker.resources]
        cpu_pcts = [r['cpu_percent'] for r in tracker.resources]
        
        plt.figure(figsize=(12, 6))
        plt.plot(times, mem_pcts, label='Memoria (%)')
        plt.plot(times, cpu_pcts, label='CPU (%)')
        plt.axhline(y=90, color='r', linestyle='--', alpha=0.7, label='Límite crítico (90%)')
        
        # Añadir marcas de checkpoints
        for cp in tracker.checkpoints:
            plt.axvline(x=cp['elapsed_total'], color='g', alpha=0.5, linestyle='-.')
        
        plt.title('Uso de recursos durante la ejecución')
        plt.xlabel('Tiempo (segundos)')
        plt.ylabel('Uso (%)')
        plt.ylim(0, 100)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{vis_dir}/resource_usage.png", dpi=300)
        plt.close()
    
    logger.info(f"Visualizaciones del proceso guardadas en {vis_dir}")

def display_log_summary():
    """Muestra un resumen del archivo de log"""
    log_file = LOG_DIR / log_filename
    if not log_file.exists():
        logger.warning(f"No se encontró el archivo de log: {log_file}")
        return
    
    # Leer últimas líneas
    try:
        with open(log_file, 'r') as f:
            lines = f.readlines()
            
        # Mostrar stats básicos
        total_lines = len(lines)
        errors = sum(1 for line in lines if " ERROR " in line)
        warnings = sum(1 for line in lines if " WARNING " in line)
        infos = sum(1 for line in lines if " INFO " in line)
        
        print(f"\n📋 Resumen del log ({log_file.name}):")
        print(f"  Total líneas: {total_lines}")
        print(f"  Información: {infos}")
        print(f"  Advertencias: {warnings}")
        print(f"  Errores: {errors}")
        
        # Mostrar últimos errores
        if errors > 0:
            print("\n⚠️ Últimos errores:")
            error_lines = [line.strip() for line in lines if " ERROR " in line]
            for line in error_lines[-min(5, len(error_lines)):]:
                print(f"  {line}")
                
    except Exception as e:
        logger.error(f"Error procesando archivo de log: {str(e)}")
        
@trace("Generación de fusión optimizada")
def generate_optimized_fusion(ceemdan_data, tvfemd_data, fusion_weights, elevation_masks):
    """
    Genera features de fusión optimizadas basadas en los pesos de importancia aprendidos
    
    Args:
        ceemdan_data: Array de características CEEMDAN (T, ny, nx, 3)
        tvfemd_data: Array de características TFV-EMD (T, ny, nx, 3)
        fusion_weights: Diccionario de pesos por nivel y componente
        elevation_masks: Diccionario de máscaras por nivel de elevación
        
    Returns:
        Array de fusión optimizada (T, ny, nx, 3)
    """
    T, ny, nx, n_components = ceemdan_data.shape
    logger.info(f"Generando fusión optimizada: shape={T}×{ny}×{nx}×{n_components}")
    
    # Inicializar array para fusión optimizada
    fusion_optimized = np.zeros_like(ceemdan_data)
    
    # Para cada nivel de elevación
    for level_name, mask in elevation_masks.items():
        level_3d_mask = np.zeros((ny, nx), dtype=bool)
        level_3d_mask = mask.reshape(ny, nx)
        
        # Para cada componente
        for comp_idx in range(n_components):
            # Obtener pesos para este nivel y componente
            if level_name in fusion_weights and fusion_weights[level_name][component_idx] is not None:
                ceemdan_weight, tvfemd_weight = fusion_weights[level_name][component_idx]
            else:
                logger.warning(f"No hay pesos para {level_name}, componente {component_idx}. Usando pesos uniformes.")
                ceemdan_weight = tvfemd_weight = 0.5
                
            # Aplicar fusión ponderada para este nivel y componente
            for t in range(T):
                # Extraer solo las celdas para este nivel
                fusion_optimized[t, level_3d_mask, comp_idx] = (
                    ceemdan_weight * ceemdan_data[t, level_3d_mask, comp_idx] +
                    tvfemd_weight * tvfemd_data[t, level_3d_mask, comp_idx]
                )
    
    logger.info(f"Fusión optimizada generada. Shape: {fusion_optimized.shape}")
    return fusion_optimized

@trace("Entrenamiento de modelo por elevación")
def train_elevation_model(level_name, mask, X_tr, Y_tr, X_va, Y_va, model_path, history_path, force_retrain=False):
    """
    Entrena un modelo BiGRU para un nivel de elevación específico o carga uno existente
    
    Args:
        level_name: Nombre del nivel de elevación
        mask: Máscara para seleccionar celdas del nivel
        X_tr: Datos de entrenamiento completos
        Y_tr: Etiquetas de entrenamiento completas
        X_va: Datos de validación completos
        Y_va: Etiquetas de validación completas
        model_path: Ruta para guardar/cargar el modelo
        history_path: Ruta para guardar/cargar el historial
        force_retrain: Si es True, se reentrenará aunque exista modelo
        
    Returns:
        Tuple: (modelo, historial, predicciones)
    """
    # Comprobar si el modelo ya existe
    if os.path.exists(model_path) and not force_retrain:
        logger.info(f"Cargando modelo BiGRU existente para nivel {level_name}...")
        try:
            model = tf.keras.models.load_model(model_path)
            
            # Intentar cargar historial
            if os.path.exists(history_path):
                history_data = np.load(history_path, allow_pickle=True)
                history = history_data['history'].item()
                logger.info(f"Historial cargado para nivel {level_name}")
            else:
                # Crear diccionario vacío si no hay historial
                history = {'loss': [], 'val_loss': []}
                
            # Generar predicciones
            logger.info(f"Generando predicciones para nivel {level_name} con modelo cargado")
            cells_in_level = np.sum(mask)
            Y_pred = predict_in_batches(model, X_va)
            
            # Reorganizar predicciones
            Y_pred_level = np.zeros((len(X_va), OUTPUT_HORIZON, cells_in_level))
            for i in range(len(X_va)):
                for h in range(OUTPUT_HORIZON):
                    Y_pred_level[i, h] = Y_pred[i, h*cells_in_level:(h+1)*cells_in_level]
            
            logger.info(f"Modelo {level_name} cargado y predicciones generadas: {Y_pred_level.shape}")
            tracker.log_metric(level_name, "loaded_model", True)
            
            return model, history, Y_pred_level
            
        except Exception as e:
            logger.error(f"Error cargando modelo {level_name}: {str(e)}")
            logger.info(f"Entrenando nuevo modelo para {level_name}...")
    else:
        logger.info(f"{'Forzando reentrenamiento' if force_retrain else 'No existe modelo'} para nivel {level_name}. Entrenando...")
    
    # Preparar datos específicos para este nivel
    cells_in_level = np.sum(mask)
    logger.info(f"Nivel {level_name}: {cells_in_level} celdas, {mask.shape}")
    
    # Extraer solo las columnas relevantes para este nivel
    X_tr_level = extract_level_features(X_tr, mask)
    Y_tr_level = extract_level_targets(Y_tr, mask)
    X_va_level = extract_level_features(X_va, mask)
    Y_va_level = extract_level_targets(Y_va, mask)
    
    logger.info(f"Shapes para {level_name}: X_tr={X_tr_level.shape}, Y_tr={Y_tr_level.shape}")
    
    # Construir y entrenar modelo BiGRU
    input_dim = X_tr_level.shape[-1]
    output_length = OUTPUT_HORIZON
    output_dim = cells_in_level
    
    # Configurar memoria limitada para evitar OOM
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logger.info("Configuración GPU aplicada para entrenamiento")
    except:
        logger.warning("No se pudo aplicar configuración específica de GPU")
    
    # Crear modelo BiGRU
    model = create_bigru_model(
        input_shape=(INPUT_WINDOW, input_dim),
        output_length=output_length,
        output_dim=output_dim,
        level_name=level_name
    )
    
    # Early stopping para prevenir overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    # Crear datasets eficientes
    batch_size = min(32, len(X_tr_level) // 10 + 1)  # Batch adaptativo
    train_dataset = create_tf_dataset(X_tr_level, Y_tr_level, batch_size)
    val_dataset = create_tf_dataset(X_va_level, Y_va_level, batch_size)
    
    # Entrenar modelo
    logger.info(f"Entrenando modelo BiGRU para nivel {level_name}...")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100,  # Número máximo de epochs
        callbacks=[early_stopping],
        verbose=1
    ).history
    
    # Guardar modelo e historial
    model.save(model_path)
    np.savez_compressed(history_path, history=history)
    logger.info(f"Modelo {level_name} guardado en {model_path}")
    
    # Generar predicciones
    logger.info(f"Generando predicciones para nivel {level_name}")
    Y_pred = predict_in_batches(model, X_va_level)
    
    # Registrar métricas
    val_mask = ~np.isnan(Y_va_level) & ~np.isnan(Y_pred)
    if np.all(val_mask):
        rmse = np.sqrt(mean_squared_error(Y_va_level, Y_pred))
        mae = mean_absolute_error(Y_va_level, Y_pred)
        r2 = r2_score(Y_va_level.reshape(-1), Y_pred.reshape(-1))
        
        tracker.log_metric(level_name, "rmse", rmse)
        tracker.log_metric(level_name, "mae", mae)
        tracker.log_metric(level_name, "r2", r2)
        logger.info(f"Modelo {level_name}: RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")
    
    return model, history, Y_pred

def extract_level_features(X, mask):
    """
    Extrae características específicas para un nivel de elevación
    
    Args:
        X: Array de características completo (samples, time_steps, features)
        mask: Máscara del nivel
    
    Returns:
        Array con características para el nivel específico
    """
    # Extraer solo columnas para este nivel
    n_samples = X.shape[0]
    time_steps = X.shape[1]
    cells_per_component = X.shape[2] // 3
    cells_in_level = np.sum(mask)
    
    # Inicializar array para características del nivel
    X_level = np.zeros((n_samples, time_steps, cells_in_level * 3))
    
    # Para cada componente
    for comp in range(3):
        # Índices de inicio y fin para este componente en array original
        start_idx = comp * cells_per_component
        end_idx = (comp + 1) * cells_per_component
        
        # Extraer solo columnas para celdas en esta elevación
        comp_features = X[:, :, start_idx:end_idx]
        comp_features_level = comp_features[:, :, mask]
        
        # Colocar en array de salida
        level_start = comp * cells_in_level
        level_end = (comp + 1) * cells_in_level
        X_level[:, :, level_start:level_end] = comp_features_level
    
    return X_level

def extract_level_targets(Y, mask):
    """
    Extrae objetivos específicos para un nivel de elevación
    
    Args:
        Y: Array de objetivos completo (samples, horizons, cells)
        mask: Máscara del nivel
    
    Returns:
        Array con objetivos para el nivel específico
    """
    n_samples = Y.shape[0]
    horizons = Y.shape[1]
    cells_in_level = np.sum(mask)
    
    # Inicializar array para objetivos del nivel
    Y_level = np.zeros((n_samples, horizons * cells_in_level))
    
    # Para cada horizonte
    for h in range(horizons):
        # Extraer datos para este horizonte
        horizon_data = Y[:, h, :]
        
        # Extraer solo celdas para este nivel
        horizon_level_data = horizon_data[:, mask]
        
        # Colocar en array de salida
        start_idx = h * cells_in_level
        end_idx = (h + 1) * cells_in_level
        Y_level[:, start_idx:end_idx] = horizon_level_data
    
    return Y_level

def create_bigru_model(input_shape, output_length, output_dim, level_name):
    """
    Crea un modelo BiGRU con estructura de autoencoder-decoder
    
    Args:
        input_shape: Tupla con forma de entrada (timesteps, features)
        output_length: Número de pasos de salida
        output_dim: Dimensión de la salida (número de celdas en el nivel)
        level_name: Nombre del nivel de elevación (para trazabilidad)
    
    Returns:
        model: Modelo BiGRU compilado
    """
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Bidirectional, GRU, TimeDistributed, Dense, Dropout, BatchNormalization
    
    model = Sequential(name=f"BiGRU_{level_name}")
    
    # Capa de entrada con ajuste de forma
    model.add(Input(shape=input_shape))
    
    # Encoder: BiGRU
    model.add(Bidirectional(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(GRU(32, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    
    # Bottleneck: capa densa para compresión
    model.add(Dense(16, activation='relu'))
    
    # Decoder: GRU unidireccional
    model.add(RepeatVector(output_length))
    model.add(GRU(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    
    # Capa de salida
    model.add(TimeDistributed(Dense(output_dim)))
    
    # Compilación del modelo
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    model.summary(print_fn=logger.info)  # Usar logger para imprimir resumen
    
    return model

# Función principal para el proceso completo
@trace("Proceso completo")
def main(force_retrain=False):
    """
    Proceso completo de entrenamiento y evaluación de TopoRain-NET
    
    Args:
        force_retrain: Si es True, fuerza el reentrenamiento de todos los modelos
        
    Returns:
        summary: Resumen final del proceso
    """
    # 1. Preparación de datos
    tracker.start_section("Preparación de datos")
    
    # Ejecutar optimización de fusión para obtener pesos y modelos
    logger.info("Optimizando fusión de CEEMDAN y TFV-EMD con XGBoost...")
    fusion_models, fusion_weights = optimize_fusion_with_xgboost(
        ceemdan_data=da_ceemdan, 
        tvfemd_data=da_tvfemd, 
        target_data=prec, 
        masks=elevation_masks,
        test_size=0.2,
        force_retrain=force_retrain
    )
    
    # Generar la fusión optimizada con los pesos aprendidos
    logger.info("Generando fusión optimizada con pesos aprendidos...")
    da_fusion_optimized = generate_optimized_fusion(
        ceemdan_data=da_ceemdan,
        tvfemd_data=da_tvfemd,
        fusion_weights=fusion_weights,
        elevation_masks=elevation_masks
    )
    
    # Registrar información sobre la fusión optimizada
    tracker.add_checkpoint("Fusión optimizada generada", {
        "shape": da_fusion_optimized.shape,
        "min": float(np.nanmin(da_fusion_optimized)),
        "max": float(np.nanmax(da_fusion_optimized)),
        "nan_count": int(np.isnan(da_fusion_optimized).sum())
    })
    
    windows_prep_start = time.time()
    
    # Preparar datos de entrada y salida con ventanas deslizantes
    logger.info(f"Creando ventanas deslizantes: INPUT_WINDOW={INPUT_WINDOW}, OUTPUT_HORIZON={OUTPUT_HORIZON}")
    
    # Obtener dimensiones
    T, ny, nx, _ = da_fusion_optimized.shape
    cells = ny * nx
    
    # Inicializar arrays para ventanas X (entrada) e Y (salida)
    N = T - INPUT_WINDOW - OUTPUT_HORIZON + 1  # Número de ventanas válidas
    
    # Inicializar arrays
    X = np.zeros((N, INPUT_WINDOW, cells * 3))  # 3 componentes de fusión por celda
    Y = np.zeros((N, OUTPUT_HORIZON, cells))
    
    # Para cada muestra
    for i in range(N):
        # Ventana de entrada: características de fusión optimizada
        for t in range(INPUT_WINDOW):
            # Reorganizar los datos: (T, ny, nx, 3) -> (T, INPUT_WINDOW, cells*3)
            X[i, t, :] = da_fusion_optimized[i+t].reshape(-1)
        
        # Ventana de salida: precipitación
        for h in range(OUTPUT_HORIZON):
            # Datos objetivo: (T, ny, nx) -> (T, OUTPUT_HORIZON, cells)
            Y[i, h, :] = prec[i+INPUT_WINDOW+h].reshape(-1)
    
    windows_prep_time = time.time() - windows_prep_start
    logger.info(f"Preparación de ventanas completada en {windows_prep_time:.2f} segundos")
    logger.info(f"Ventanas válidas totales: {N}")
    
    # Escalado de features
    scale_start = time.time()
    logger.info("Escalado de features...")
    scX = StandardScaler()
    Xf = scX.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    scale_time = time.time() - scale_start
    
    # Train/val split
    split = int(0.7*N)
    X_tr = Xf[:split]
    X_va = Xf[split:]
    Y_tr = Y[:split]
    Y_va = Y[split:]
    logger.info(f"Split train={len(X_tr)}, val={len(X_va)}")
    
    tracker.end_section()
    tracker.add_checkpoint("Ventanas y split preparados", {
        "total_windows": N,
        "train_size": len(X_tr),
        "val_size": len(X_va),
        "window_prep_time": windows_prep_time
    })
    
    # 4. Entrenar modelos por nivel de elevación
    tracker.start_section("Entrenamiento de modelos por nivel")
    elevation_models = {}
    elevation_histories = {}
    elevation_predictions = {}

    for level_name, mask in elevation_masks.items():
        model_path = TRAINED_DIR / f"BiGRU_{level_name}_model.keras"
        history_path = HISTORY_DIR / f"{level_name}_history.npz"
        
        # Entrenar o cargar modelo para este nivel, respetando force_retrain
        model, history, Y_pred_level = train_elevation_model(
            level_name, mask, X_tr, Y_tr, X_va, Y_va, model_path, history_path, force_retrain
        )
        
        # Almacenar resultados
        elevation_models[level_name] = model
        elevation_histories[level_name] = history
        elevation_predictions[level_name] = Y_pred_level
    
    tracker.end_section()
    tracker.add_checkpoint("Modelos por nivel entrenados", {
        "num_models": len(elevation_models),
        "models": list(elevation_models.keys())
    })
    
    # 5. Meta-modelo de fusión
    tracker.start_section("Meta-modelo de fusión")
    
    # Cargar o entrenar meta-modelo
    meta_model_path = TRAINED_DIR / "meta_fusion_model.pkl"
    meta_preds_path = PRED_DIR / "meta_fusion_preds.npz"

    # Inicializar la variable Y_meta_va
    Y_meta_va = None
    meta_model_loaded = False

    # Intentar cargar meta-modelo y predicciones desde disco si no se fuerza reentrenamiento
    if os.path.exists(meta_model_path) and os.path.exists(meta_preds_path) and not force_retrain:
        logger.info("Cargando meta-modelo y predicciones existentes...")
        load_start = time.time()
        try:
            meta_model, meta_info = load_model('meta', 'all')
            
            # Cargar predicciones
            meta_preds_data = np.load(meta_preds_path)
            Y_meta_va = meta_preds_data['predictions']
            
            load_time = time.time() - load_start
            logger.info(f"Meta-modelo y predicciones cargados correctamente en {load_time:.2f} segundos")
            
            tracker.log_metric("meta_modelo", "loaded", True)
            tracker.log_metric("meta_modelo", "load_time", load_time)
            meta_model_loaded = True
            
        except Exception as e:
            logger.error(f"Error cargando meta-modelo o predicciones: {str(e)}")
            logger.warning("Entrenando nuevo meta-modelo...")
            meta_model_loaded = False
    
    @trace("Construcción de meta-modelo")
    def build_meta_fusion_model(base_preds, Y_true):
        """
        Construye un meta-modelo que fusiona las predicciones de los modelos base
        
        Args:
            base_preds: Diccionario {nombre_modelo: predicciones}
                        donde predicciones tiene forma (samples, horizons, cells)
            Y_true: Valores reales para entrenamiento (samples, horizons, cells)
            
        Returns:
            meta_model: Modelo entrenado
            X_meta: Características de entrada para meta-modelo
            Y_meta_pred: Predicciones del meta-modelo
        """
        logger.info(f"Construyendo meta-modelo con {len(base_preds)} modelos base")
        n_samples = next(iter(base_preds.values())).shape[0]
        n_horizons = next(iter(base_preds.values())).shape[1]
        n_cells = next(iter(base_preds.values())).shape[2]
        
        # Crear meta-modelo para cada horizonte de predicción
        meta_models = []
        
        # Preparar datos para meta-modelo
        X_meta = []
        for h in range(n_horizons):
            # Para cada horizonte, concatenar predicciones de todos los modelos
            X_h = []
            for model_name, preds in base_preds.items():
                X_h.append(preds[:, h, :])
            
            # Concatenar todas las predicciones para este horizonte
            if X_h:
                X_meta.append(np.hstack(X_h))
        
        # Entrenamiento de un meta-modelo por horizonte
        logger.info(f"Entrenando {n_horizons} meta-modelos XGBoost para fusión")
        
        # Parámetros para modelos meta-XGBoost
        xgb_params = {
            'tree_method': 'hist',
            'n_estimators': 100,
            'max_depth': 5,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'n_jobs': min(4, os.cpu_count() - 1)
        }
        
        # Entrenar modelos por horizontes (paralelizable)
        for h in range(n_horizons):
            logger.info(f"  Entrenando meta-modelo para horizonte {h+1}/{n_horizons}")
            
            # Extraer datos para este horizonte
            X_h = X_meta[h]
            Y_h = Y_true[:, h, :].reshape(n_samples, -1)
            
            # Verificar datos
            if np.isnan(X_h).any() or np.isnan(Y_h).any():
                logger.warning(f"Detectados NaN en los datos para horizonte {h}. Realizando imputación.")
                X_h = np.nan_to_num(X_h, nan=0.0)
                Y_h = np.nan_to_num(Y_h, nan=0.0)
            
            # Train-test split para este horizonte
            X_h_train, X_h_test, Y_h_train, Y_h_test = train_test_split(X_h, Y_h, test_size=0.2, random_state=42)
            
            # Entrenar modelo XGBoost para este horizonte
            model_h = train_xgb_with_memory_optimization(X_h_train, Y_h_train, X_h_test, Y_h_test, xgb_params)
            meta_models.append(model_h)
            
            # Evaluar modelo
            y_h_pred = predict_xgb_in_batches(model_h, X_h_test)
            rmse = np.sqrt(mean_squared_error(Y_h_test.ravel(), y_h_pred))
            r2 = r2_score(Y_h_test.ravel(), y_h_pred)
            logger.info(f"  Meta-modelo horizonte {h+1}: RMSE={rmse:.4f}, R²={r2:.4f}")
            
            # Registrar métricas
            tracker.log_metric(f"meta_modelo_h{h+1}", "rmse", rmse)
            tracker.log_metric(f"meta_modelo_h{h+1}", "r2", r2)
        
        # Generar predicciones meta-modelo
        Y_meta_pred = np.zeros((n_samples, n_horizons, n_cells))
        
        logger.info("Generando predicciones del meta-modelo...")
        for h in range(n_horizons):
            Y_meta_pred[:, h, :] = predict_xgb_in_batches(meta_models[h], X_meta[h]).reshape(n_samples, n_cells)
        
        # Guardar meta-modelo y predicciones
        save_start = time.time()
        
        # Guardar modelo y predicciones
        meta_info = {
            'training_date': datetime.datetime.now().strftime(timestamp_format),
            'base_models': list(base_preds.keys()),
            'horizons': OUTPUT_HORIZON,
            'input_shape': np.array(X_meta).shape
        }
        save_model(meta_models, 'meta', 'all', extra_info=meta_info)
        
        # Guardar predicciones
        np.savez_compressed(meta_preds_path, predictions=Y_meta_pred)
        save_time = time.time() - save_start
        logger.info(f"Meta-modelo y predicciones guardados en {save_time:.2f} segundos")
        
        return meta_models, X_meta, Y_meta_pred
    
    # Si no se cargó correctamente o se fuerza reentrenamiento, entrenar nuevo modelo
    if not meta_model_loaded or force_retrain:
        logger.info("Entrenando nuevo meta-modelo...")
        # Construir predicciones de modelos base para entrenamiento
        base_preds = {}
        for level_name, mask in elevation_masks.items():
            if level_name in elevation_predictions:
                # Reconstruir predicciones completas
                preds = elevation_predictions[level_name]
                complete_preds = np.zeros((len(X_va), OUTPUT_HORIZON, cells))
                
                for i in range(len(X_va)):
                    for h in range(OUTPUT_HORIZON):
                        complete_preds[i, h, mask] = preds[i, h]
                
                base_preds[f"BiGRU_{level_name}"] = complete_preds
            
        # Entrenar meta-modelo con predicciones y valores reales
        if base_preds:
            meta_model, X_meta, Y_meta_va = build_meta_fusion_model(base_preds, Y_va)
        else:
            logger.warning("No hay suficientes modelos base para entrenar meta-modelo")
            meta_model = None
            
    # 6. Evaluación completa
    tracker.start_section("Evaluación de modelos")
    logger.info("Ejecutando evaluación completa...")
    
    # Reconstruir predicciones completas para cada nivel
    elevation_preds_complete = {}
    
    for level_name, mask in elevation_masks.items():
        if level_name not in elevation_predictions:
            logger.warning(f"No hay predicciones para el nivel {level_name}. Omitiendo.")
            continue
            
        preds = elevation_predictions[level_name]
        complete_preds = np.zeros((len(X_va), OUTPUT_HORIZON, cells))
        
        for i in range(len(X_va)):
            for h in range(OUTPUT_HORIZON):
                complete_preds[i, h, mask] = preds[i, h]
        
        elevation_preds_complete[f"BiGRU-{level_name}"] = complete_preds
    
    # Añadir meta-modelo a las predicciones
    all_predictions = elevation_preds_complete.copy()
    
    if Y_meta_va is not None:
        all_predictions["Meta-Fusion"] = Y_meta_va
    
    # Verificar que hay predicciones para evaluar
    if not all_predictions:
        logger.error("No hay predicciones disponibles para evaluar. Abortando.")
    else:
        # Métricas globales
        metrics_global = calculate_global_metrics(all_predictions, Y_va)
        metrics_global.to_csv(f"{BASE}/models/output/elevation_models_metrics.csv", index=False)
        
        # Registrar métricas globales para trazabilidad
        for _, row in metrics_global.iterrows():
            model_name = row['Model']
            for metric in ['MAE', 'RMSE', 'MAPE', 'R²']:
                if metric in row:
                    tracker.log_metric(f"global_{model_name}", metric.lower(), row[metric])
        
        # Métricas por niveles de elevación
        metrics_elevation = calculate_metrics_by_elevation(all_predictions, Y_va, elevation_masks)
        metrics_elevation.to_csv(f"{BASE}/models/output/metrics_by_elevation_detailed.csv", index=False)
        
        # Métricas por percentiles
        percentiles = [0, 50, 90, 95, 99]
        metrics_percentile = calculate_metrics_by_percentiles(all_predictions, Y_va, percentiles)
        metrics_percentile.to_csv(f"{BASE}/models/output/metrics_by_percentile.csv", index=False)
        
        # Generar visualizaciones
        logger.info("Generando visualizaciones...")
        
        # Mapas de predicción para cada horizonte
        for h in range(OUTPUT_HORIZON):
            plot_all_model_maps(all_predictions, Y_va, lat, lon, example_idx=0, horizon_idx=h)
        
        # Scatter plots
        compare_models_scatter(all_predictions, Y_va)
        
        # Gráficos de barras de métricas
        for metric in ['MAE', 'RMSE', 'MAPE']:
            plot_metrics_comparison(metrics_global, metric=metric)
            plot_metrics_by_elevation(metrics_elevation, metric=metric)
            plot_metrics_by_percentiles(metrics_percentile, metric=metric)
    
    tracker.end_section()
    
    # Generar resumen final del proceso
    summary = tracker.summary()
    
    return summary
# Asegurarnos de que todos los directorios necesarios existen
for directory in [LOG_DIR, TRAINED_DIR, PRED_DIR, HISTORY_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"Directorio asegurado: {directory}")

# Ejecutar el proceso completo con manejo de excepciones y trazabilidad mejorada
if __name__ == "__main__":
    try:
        # Analizar argumentos de línea de comandos (si los hay)
        import argparse
        parser = argparse.ArgumentParser(description="Entrenamiento de TopoRain-NET")
        parser.add_argument('--force-retrain', action='store_true', 
                          help='Forzar reentrenamiento de todos los modelos')
        
        # En notebooks podemos capturar los argumentos si se ejecuta como script
        try:
            args = parser.parse_args()
            force_retrain = args.force_retrain
        except:
            # Si falla (por ejemplo, en ejecución de notebook interactivo)
            force_retrain = False
        
        logger.info(f"🚀 Iniciando proceso TopoRain-NET{'(forzando reentrenamiento)' if force_retrain else ''}")
        process_summary = main(force_retrain=force_retrain)
        logger.info("✅ Proceso TopoRain-NET completado exitosamente")
        # Visualizar resultados de trazabilidad
        visualize_process_tracker_results()
        # Mostrar resumen del log
        display_log_summary()
    except Exception as e:
        logger.error(f"❌ Error en el proceso principal: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        try:
            tracker.summary()  # Generar resumen incluso si hay error
        except Exception as summary_error:
            logger.error(f"Error al generar resumen: {str(summary_error)}")
        raise

# Asegurarnos de que todos los directorios necesarios existen
for directory in [LOG_DIR, TRAINED_DIR, PRED_DIR, HISTORY_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"Directorio asegurado: {directory}")

# Ejecutar el proceso completo con manejo de excepciones y trazabilidad mejorada
if __name__ == "__main__":
    try:
        logger.info("🚀 Iniciando proceso TopoRain-NET...")
        process_summary = main()
        logger.info("✅ Proceso TopoRain-NET completado exitosamente")
        # Visualizar resultados de trazabilidad
        visualize_process_tracker_results()
        # Mostrar resumen del log
        display_log_summary()
    except Exception as e:
        logger.error(f"❌ Error en el proceso principal: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        try:
            tracker.summary()  # Generar resumen incluso si hay error
        except Exception as summary_error:
            logger.error(f"Error al generar resumen: {str(summary_error)}")
        raise

2025-05-29 13:05:32,954 [INFO] Configuración de threading de TensorFlow aplicada
Entorno configurado. Usando ruta base: ..
2025-05-29 13:05:32,957 [INFO] Cargando datasets y separando características CEEMDAN y TFV-EMD...
Entorno configurado. Usando ruta base: ..
2025-05-29 13:05:32,957 [INFO] Cargando datasets y separando características CEEMDAN y TFV-EMD...
2025-05-29 13:05:33,127 [INFO] Clusters codificados de texto a números: {'high': 0, 'low': 1, 'medium': 2}
2025-05-29 13:05:33,128 [INFO] Dimensiones: T=530, ny=61, nx=65, cells=3965
2025-05-29 13:05:33,129 [INFO] Shapes: prec=(530, 61, 65), da_ceemdan=(530, 61, 65, 3), da_tvfemd=(530, 61, 65, 3)
2025-05-29 13:05:33,129 [INFO] Definiendo máscaras para los niveles de elevación...
2025-05-29 13:05:33,129 [INFO] Distribución de celdas por nivel de elevación:
2025-05-29 13:05:33,130 [INFO]   Nivel 1 (<957m): 2048 celdas
2025-05-29 13:05:33,130 [INFO]   Nivel 2 (957-2264m): 921 celdas
2025-05-29 13:05:33,131 [INFO]   Nivel 3 (>2264m): 9

usage: ipykernel_launcher.py [-h] [--force-retrain]
ipykernel_launcher.py: error: argument --force-retrain: ignored explicit argument '/Users/riperez/Library/Jupyter/runtime/kernel-v319d7d8c1b7a304bdfb8b46549569819f7d24cb0e.json'


2025-05-29 13:05:33,667 [INFO] ▶️ INICIANDO: Preparación de datos
2025-05-29 13:05:33,769 [INFO] Optimizando fusión de CEEMDAN y TFV-EMD con XGBoost...
2025-05-29 13:05:33,769 [INFO] ✓ COMPLETADO: Preparación de datos en 0.10 segundos
2025-05-29 13:05:33,769 [INFO] Optimizando fusión de CEEMDAN y TFV-EMD con XGBoost...
2025-05-29 13:05:33,769 [INFO] ✓ COMPLETADO: Preparación de datos en 0.10 segundos
2025-05-29 13:05:33,875 [INFO] ▶️ INICIANDO: Optimización de fusión
2025-05-29 13:05:33,875 [INFO] ▶️ INICIANDO: Optimización de fusión

🖥️  Recursos detectados: 10 CPUs, 16.0GB RAM (2.9GB disponible)
🔧 Configuración optimizada: 8 workers en paralelo FORZADOS, tree_method=hist
🧠 Memoria disponible: 2.88GB (82.0% usado)

📊 Iniciando entrenamiento acelerado de 9 componentes (3 niveles × 3 componentes)

⚡ Activando procesamiento paralelo forzado con 8 workers para acelerar el entrenamiento
▶️  Nivel nivel_1, componente 0: Iniciando entrenamiento rápido...
▶️  Nivel nivel_1, componente 1: Inic